***Import needed libraries***

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import multiprocessing
from multiprocessing import Pool
import regex

**The databases that the code needs during the work:**         
1) Key_indications.csv         
2) CLINICALTRIALS_ORIGINAL.csv          
3) EFO2MESH.csv       
4) NCTID_CONDITION_EFO.csv        
5) NCT_ID2MOLREGNO.csv        
6) EFO2NAME.csv       
7) INTERVENTIONS.csv       
8) TRIALS_TERMITE.csv      
9) TERMITE_TABLE.csv      
10) Outcome.csv        
11) NOT_MATCHED_WORDS.csv       
12) SYNONYMS_FULL_CL.csv        
13) tweets_db.csv         
14) Phases.csv         

# <font color='green'> I. Exctracting information </font>

Export the needed table with whole tweets, delete the columns that are not needed.       
i_table ~ initial table

In [2]:
#create a table with names of columns
i_table = pd.read_csv(r'tweets_db.csv', encoding='utf8', names=['TWID', 'USER_NAME', 'SCREEN_NAME', 'TWEET_ID', 
                                                                'TWEET', 'TWEET_URL', 'TWEET_CREATED_AT', 
                                                                'QUERY_METHOD', 'QUERY_USED', 'INSERTED_AT'])

#deleting not necessary columns
del i_table['USER_NAME']
del i_table['SCREEN_NAME']
del i_table['TWEET_URL']
del i_table['TWEET_CREATED_AT']
del i_table['QUERY_METHOD']
del i_table['QUERY_USED']
del i_table['INSERTED_AT']

print('The length of the initial table =' + ' ' + str(len(i_table)))
i_table[:3]

The length of the initial table = 15897


,TWID,TWEET_ID,TWEET
0,1,986637456241029123,Session on Brexit implications at #DIAEurope20...
1,2,986605359111565312,Patient Centricity Beyond the Talk session at ...
2,3,986546919467085825,The FDA approved Rigel's Tavalisse for chronic...


Create new working table where will be relevant tweets from the initial table.

In [3]:
w_table = pd.DataFrame(columns=['TWID', 'TWEET_ID', 'TWEET']) #create new table
w_table

,TWID,TWEET_ID,TWEET


In [4]:
#fill in the new table with relevant tweets

pos = 0 #the position in w_table
for i in range(len(i_table)):
    trial = re.findall(r'trial', str(i_table.loc[i, 'TWEET']).lower()) #looking for 'trial' in tweet
    phase = re.findall(r'phase', str(i_table.loc[i, 'TWEET']).lower()) #looking for 'phase' in trial
    ph = re.findall(r'ph\s?\d?\w?\s', str(i_table.loc[i, 'TWEET']).lower()) #looking for 'ph ' in trial
    #if selected words are in tweets we add them into our w_table
    if len(trial) > 0 or len(phase) > 0 or len(ph) > 0:
        w_table.at[pos, 'TWID'] = i_table.loc[i, 'TWID']
        w_table.at[pos, 'TWEET_ID'] = i_table.loc[i, 'TWEET_ID']
        w_table.at[pos, 'TWEET']= i_table.loc[i, 'TWEET']
        pos += 1

In [5]:
print('Lenght of working table is', len(w_table))

Lenght of working table is 1131


So we work with the table that looks like this ->

In [6]:
w_table[:5]

,TWID,TWEET_ID,TWEET
0,4,986398045897191424,@PoseidaThera present promising results in ini...
1,16,986219643881885698,Genentech gains a license option to Kineta's a...
2,21,985928258708492288,First-in-human trial with MG7-CART shows effic...
3,33,985172281734377473,Roche's CpAM modulator RO7049389 well tolerate...
4,40,984750427186126848,Live at #ILC2018 FXR agonist EYP001 from Enyo ...


## <font color='green'> Phases </font>

Now we add a column and information about the phases. Running through a table with variants of phases and finding the needed ones.      
There are some tweets where is information more than about one phase. Make a column with number of phases.

In [7]:
phases = pd.read_csv('Phases.csv', encoding='utf8') #open a dataframe with variants of phases
del phases['Unnamed: 0']

In [8]:
phases[:4]

,Variant,Outcome
0,phase 1\s,Phase 1
1,phase 2\s,Phase 2
2,phase 3\s,Phase 3
3,phase I\s,Phase 1


In [9]:
w_table['PHASE'] = 'Null'
w_table['N_PHASES'] = 'Null'

In [10]:
#Find a phase for each tweet
for i in range(len(w_table)):
    for j in range(len(phases)):
        #delete from tweet all the mess
        line = str(w_table.loc[i, 'TWEET']).lower().replace('.', ' ').replace(',', ' ').replace('?', ' ').replace(';', ' ')
        res = re.findall(str(phases.loc[j, 'Variant']).lower(), line) #search a phase in tweet
        if len(res) > 0: #if there is - pick it in a list
            w_table.at[i, 'PHASE'] = phases.loc[j, 'Outcome'] # Outcome is a column with the exact form of the phase (ex. Phase 1) 

In [11]:
#count how many cells are without phases
null_phases = 0 #counter
for i in range(len(w_table)):
    if str(w_table.loc[i, 'PHASE']) == 'Null': #if there is no phase
        null_phases += 1 #add one to counter
        
print("The amount tweets without information about phases is", null_phases)

The amount tweets without information about phases is 442


In [12]:
#count how many phases are in the row
#make a separate column with number of phases (N_PHASES)
for i in range(len(w_table)):
    ph = re.findall(r'\s\\ph\s', w_table.loc[i, 'TWEET'].lower()) #if there is 'ph'
    ph_d = re.findall(r'ph\d', w_table.loc[i, 'TWEET'].lower()) #if there is 'ph1' or 'ph2' or 'ph3'
    p_d = re.findall(r'p(?:i|ii|iii|1|2|3)', w_table.loc[i, 'TWEET'].lower()) #if there is 'p1' or 'pIII' etc
    phase = re.findall(r'phase', w_table.loc[i, 'TWEET'].lower()) #if there is simple 'phase'
    first_in_tr = re.findall(r'first', w_table.loc[i, 'TWEET'].lower()) #if there is 'first' (first-in-trial, etc)
    preclinical_tr = re.findall(r'preclinical trial', w_table.loc[i, 'TWEET'].lower()) #if there is 'preclinical trial'
    # the number of phases is the sum of what was found ^
    w_table.at[i, 'N_PHASES'] = len(phase) + len(ph) + len(ph_d) + len(p_d) + len(first_in_tr) + len(preclinical_tr)
    
    #this line is just for other variants
    #if there is already found phase by table but not found by this cell just add 1
    if w_table.loc[i, 'N_PHASES'] == 0 and w_table.at[i, 'PHASE'] != 'Null':       
        w_table.at[i, 'N_PHASES'] = 1

In [13]:
w_table[:2]

,TWID,TWEET_ID,TWEET,PHASE,N_PHASES
0,4,986398045897191424,@PoseidaThera present promising results in ini...,Phase 1,1
1,16,986219643881885698,Genentech gains a license option to Kineta's a...,Null,0


## <font color='green'> Synonyms and MOLREGNOs </font>

In [14]:
#create new columns with information about found synonyms and molregno
w_table['SYNONYMS'] = 'Null'
w_table['MOLREGNO'] = 'Null'

Import full base of synonyms. During the work we found some mistakes in this table. There were synonyms like today, it, we should delete them.

In [15]:
synonyms = pd.read_csv(r'SYNONYMS_FULL_CL.csv', encoding='utf8', names = ['MOLREGNO', 'SYNONYMS', 'SYN_TYPE', 'MOLSYN_ID', 'RES_STEM_ID', 'MOLECULE_SYNONYM'])
synonyms[:3]

,MOLREGNO,SYNONYMS,SYN_TYPE,MOLSYN_ID,RES_STEM_ID,MOLECULE_SYNONYM
0,406801,10-METHOXYLUOTONIN A,TRADE_NAME_OLD,409.0,NaN,10-Methoxyluotonin A
1,371540,10-METHOXYMETHYLOXY-GINKGOLIDE A,TRADE_NAME_OLD,410.0,NaN,10-Methoxymethyloxy-Ginkgolide A
2,510072,10-O-METHYLJERANTININE A,TRADE_NAME_OLD,411.0,NaN,10-O-Methyljerantinine A


In [16]:
print('Length of the initial table with synonyms', len(synonyms))

Length of the initial table with synonyms 661425


In [17]:
#cleaning the synonyms table
mistakes = ['^it$', 'today', 'target', '^dry clear', 'good health', '^at iii$', '^go$', '^may$', 'stop', 'good health'] #list of not relevant for text mining synonyms
drop_mistakes = [] #list for indexes to delete

for i in range(len(synonyms)): 
    for j in range(len(mistakes)):
        mistake = re.findall(mistakes[j], str(synonyms.loc[i, 'SYNONYMS']).lower()) #look for mistake in synonyms table
        if len(mistake) > 0: #if there is
            drop_mistakes.append(i) #add the index to the list
        
synonyms = synonyms.drop(synonyms.index[drop_mistakes]) #drop not relevant synonyms
synonyms.index = np.arange(len(synonyms)) #update indexes
print('Length of clean table with synonyms', len(synonyms))

Length of clean table with synonyms 661385


### <font color='green'>Match synonyms with tweets </font>

In [20]:
#Parallelization of the process, making 6 chunks to work with them on different kernels
#Make 6 parts of the working table

a = w_table[0 : len(w_table) // 6]
a.index = np.arange(len(a))

b = w_table[len(w_table) // 6 : len(w_table) // 3]
b.index = np.arange(len(b))

c = w_table[len(w_table) // 3 : len(w_table) // 2]
c.index = np.arange(len(c))

d = w_table[len(w_table) // 2 : (2 * len(w_table)) // 3]
d.index = np.arange(len(d))

e = w_table[(2 * len(w_table)) // 3 : (5 * len(w_table)) // 6]
e.index = np.arange(len(e))

f = w_table[(5 * len(w_table)) // 6 : len(w_table)]
f.index = np.arange(len(f)) 

#list of chunks of the working table that will be run by function
chunks = [a, b, c, d, e, f]

#number of chunks = number of kernels that is going to use during the process

In [21]:
def find_synonyms(chunk): #function that is used in parallezation and is looking for the synonyms
    chunk_copy = chunk #copy to use its' length to work on
    for i in range(len(chunk_copy)):
        for j in range(len(synonyms)):
            #clean synonyms from (+) and so on
            syn = synonyms.loc[j, 'SYNONYMS'].lower().replace('+', '').replace(')', '').replace('(', '').replace('[', '').replace(']', '').replace('|', '').replace(',', '').replace('.', '').replace(':', '').replace(';', '').replace('{', '').replace('}', '')
            #clean tweet from everything that could prevent matching
            tweet = chunk.loc[i, 'TWEET'].lower().replace('?', '').replace('!', '').replace('\\', '').replace("'", '').replace('@', ' ').replace('-', '').replace('+', ' ').replace('#', ' ').replace(')', ' ').replace('(', ' ').replace('[', ' ').replace(']', ' ').replace('|', ' ').replace(',', ' ').replace('.', ' ').replace(':', ' ').replace(';', ' ').replace('{', ' ').replace('}', ' ').split()
            syn_split = syn.split() #split the word from table with synonyms (to look how many words are in it)
            
            if len(syn_split) == 1: #if it is one word
                if syn.lower() in tweet: #if synonym is in tweet
                    if str(chunk.loc[i, 'SYNONYMS']) == 'Null': #if synonyms' cell is empty add the information
                        chunk.at[i, 'SYNONYMS'] = synonyms.loc[j, 'SYNONYMS'].upper() 
                        chunk.at[i, 'MOLREGNO'] = synonyms.loc[j, 'MOLREGNO']
                    elif str(chunk.loc[i, 'SYNONYMS']) != 'Null': #if synonyms' cell is full
                        new_row = chunk.iloc[i] #create a new row
                        chunk = chunk.append(new_row) #add it to the end of the table
                        chunk.index = np.arange(len(chunk)) #update the indices
                        chunk.at[len(chunk) - 1, 'SYNONYMS'] = synonyms.loc[j, 'SYNONYMS'].upper() #add the synonym to new row
                        chunk.at[len(chunk) - 1, 'MOLREGNO'] = synonyms.loc[j, 'MOLREGNO'] #add the molregno to new row 

            c = 0 #counter       
            if len(syn_split) > 1: #if there is more than one word in synonym
                for k in range(len(syn_split)): #check if every word is in tweet
                    if syn_split[k] in tweet: #if it is in tweet add 1 to counter
                        c += 1 
                if c == len(syn_split): #if counter is the same as length of synonym
                    if str(chunk.loc[i, 'SYNONYMS']) == 'Null': #if synonyms' cell is empty add the information
                        chunk.at[i, 'SYNONYMS'] = synonyms.loc[j, 'SYNONYMS'].upper()
                        chunk.at[i, 'MOLREGNO'] = synonyms.loc[j, 'MOLREGNO']
                    elif str(chunk.loc[i, 'SYNONYMS']) != 'Null': #if synonyms' cell is full
                        new_row = chunk.iloc[i] #create a new row
                        chunk = chunk.append(new_row) #add it to the end of the table
                        chunk.index = np.arange(len(chunk)) #update the indices
                        chunk.at[len(chunk) - 1, 'SYNONYMS'] = synonyms.loc[j, 'SYNONYMS'].upper() #add the synonym to new row
                        chunk.at[len(chunk) - 1, 'MOLREGNO'] = synonyms.loc[j, 'MOLREGNO'] #add the molregno to new row              
    return chunk


if __name__ == '__main__': #constructor for parallezation
    p = Pool(6) #use 6 kernels to work on them
    result = p.map(find_synonyms, chunks) #
    #print(result)   #prints the list with updated chunks
    
    
dataframe = w_table[0:0] #make the frame for the main table
for i in range(len(result)):
        dataframe = dataframe.append(result[i]) #add parts to the main table
        
        
dataframe = dataframe.drop_duplicates(subset=['TWID', 'SYNONYMS', 'MOLREGNO'], keep='first', inplace=False) #delete all duplicates
dataframe = dataframe.sort_values(by = ['TWID']) #sort by twids
dataframe.index = np.arange(len(dataframe)) #update the indexes

In [23]:
print('Length of dataframe now is increased. Now is', len(dataframe))

Length of dataframe now is increased. Now is 1224


In [24]:
dataframe[:10]

,TWID,TWEET_ID,TWEET,PHASE,N_PHASES,SYNONYMS,MOLREGNO
0,4,986398045897191424,@PoseidaThera present promising results in ini...,Phase 1,1,Null,Null
1,16,986219643881885698,Genentech gains a license option to Kineta's a...,Null,0,ACETYLCHOLINE,27812
2,16,986219643881885698,Genentech gains a license option to Kineta's a...,Null,0,ACETYLCHOLINE,195974
3,21,985928258708492288,First-in-human trial with MG7-CART shows effic...,Phase 1,1,Null,Null
4,33,985172281734377473,Roche's CpAM modulator RO7049389 well tolerate...,Phase 1,1,Null,Null
5,40,984750427186126848,Live at #ILC2018 FXR agonist EYP001 from Enyo ...,Phase 1,1,Null,Null
6,65,981919705912004609,Boehringer Ingelheim to progress OSE's SIRP-al...,Null,0,Null,Null
7,106,977514137046667265,41 patients have been enrolled in the ongoing ...,Phase 2,1,BINIMETINIB,1677014
8,106,977514137046667265,41 patients have been enrolled in the ongoing ...,Phase 2,1,RIBOCICLIB,2039237
9,106,977514137046667265,41 patients have been enrolled in the ongoing ...,Phase 2,1,RIBOCICLIB,1926959


In [26]:
#to save it to csv for some reason
dataframe.to_csv('Working_dataframe_03.csv', sep='\t', encoding='utf8')

#### <font color='indigo'> *Important information about matchings after this process* </font>

In [27]:
# The cell to make some conslusion about matching
c = 0 
indexes_synonyms = ['0'] #the list with indexes of rows where synonyms were added by the table of synonyms

for i in range(len(dataframe)):
    if (str(dataframe.loc[i, 'SYNONYMS']) != 'Null'):
        c += 1

for i in range(len(dataframe)): 
    if (str(dataframe.loc[i, 'SYNONYMS']) != 'Null') and (str(dataframe.loc[i, 'TWID']) not in indexes_synonyms):
        indexes_synonyms.append(str(dataframe.loc[i, 'TWID']))

uni = 0 #count how many unique twids there are
for i in range(1, len(dataframe)):
    if str(dataframe.loc[i, 'TWID']) != str(dataframe.loc[i - 1, 'TWID']):
        uni += 1
        
print('The amount of synonyms that was added with this table is', c)
print('The length of the dataframe is', len(dataframe))
print('___________')
print('The amount of unique tweets that now has synonyms is', len(indexes_synonyms) - 1)
print('The amount of unique tweets is now', uni)

The amount of synonyms that was added with this table is 400
The length of the dataframe is 1224
___________
The amount of unique tweets that now has synonyms is 307
The amount of unique tweets is now 1130


In [28]:
#table_with_synonyms_first = dataframe

In [29]:
dataframe[:2]

,TWID,TWEET_ID,TWEET,PHASE,N_PHASES,SYNONYMS,MOLREGNO
0,4,986398045897191424,@PoseidaThera present promising results in ini...,Phase 1,1,Null,Null
1,16,986219643881885698,Genentech gains a license option to Kineta's a...,Null,0,ACETYLCHOLINE,27812


### <font color='green'> Matching with the list of acronyms (new column) </font>

In [30]:
# Names of trials I found:
trials = ['HMAXIK', 'APECS', 'BRIGHTE', 'CONTI-PV', 'BISE', 'LMTX', 'COPD', 'APEX', 'AML', 'CAR', 'ADXS-NEO', 
          'EFFECT-HF', 'AEGIS', 'DLBCL', 'NASH', 'PoC', 'PTGX', 'MPS', 'MATCH', 'ASND', 'GPCR', 'CART', 'SWOG',
          'KEYNOTE']

dataframe['ACRONYM'] = 'Null' #make a new column to fill there the information about acronyms
from_acr = []
how_many_acr = 0 #counter
for i in range(len(dataframe)):
    if str(dataframe.loc[i, 'SYNONYMS']) == 'Null': #if the information about synonyms is not found
        #clean the tweet and look there for a acronym
        tweet = dataframe.loc[i, 'TWEET'].replace('@', ' ').replace('$', ' ').replace('#', ' ').replace('?', ' ').replace('!', ' ').replace('\\', ' ').replace('+', ' ').replace(')', ' ').replace('(', ' ').replace('[', ' ').replace(']', ' ').replace('|', ' ').replace(',', ' ').replace('.', ' ').replace(':', ' ').replace(';', ' ').replace('{', ' ').replace('}', ' ').split()
        for j in range(len(trials)):
            if trials[j] in tweet: #if trial in tweet
                dataframe.at[i, 'ACRONYM'] = trials[j] #add it to a column with acronyms
                how_many_acr += 1 #counter + 1
                from_acr.append(i) #to know where acronyms were added
                
print('Added information to', how_many_acr, 'tweets by trials list') 

Added information to 42 tweets


### <font color='green'> Matching with regular expressions </font>

In [31]:
#list of things I don't want to match by reg_exp, some mess
notl = ['vision2016', 'First-in', 'first-in', '11', 'ucel-T', 'icense2016', 'Nonfiction8879', 'califf001', 'bluejaunt100', 'S-like', 'icrobe2018', 'anut-aller', 'muno-oncol', 'sing-remit', '99-100', 'ayed-relea', 
        'Post-hoc',  'EA2017', 'umab-EU', 'ILC2018', 'oAsia2018', 'EAU18', 'Editor-in', 'ASH2017', 'Paris2017', 'upYby68',
        'KhUtk42', 'osis-relat', '4Q17', '9-39', 'Aiims1742', 'non-valvu', 'raft-versu', 'owKWW16', 'rgets17', 'EULAR2018', 'ASCO18', '2017', '2018', '2016', '2015',
        'shwin97', 'itor-refra', 'IB-MS', 'CD33-speci', 'aliff001', 'roof-of', 'Chi-Med' 'IWD2018', 'ODlp78', 'ndera07', 'jaunt100', 'ction8879', 'Acc18', 'AACR2018',
        'ESMO2017', 'power130', 'Chi-Med', 'EBV-speci', 'HFE-relat', 'asco18', 'gmt9992', 'power150', 'NOP-46', 'rcFq50', '1000', '1539', 'ASCO17', 
        '4Q2017', 'T1E28', 'anti-P', 'ADXS-NEO', 'HIF-PH', '1Q18', 'RION-1', 'AHA16', 'anti-cance', '2609', 'DWeek2016', 'EAU17', 'qtifA46', 'Zr89', '12019', 
        'unCD47', 'ash17', 'dime-aviba', 'ntra-abdom', 'eO94', 'JPM18', 'ARVO2018', 'OVWLv30', 'CIMT2018', 'SBS2018', 'gx17', 'b0HKd44', 
        'l2ovv62', 'CCMID2018', '36-51', 'crobe2018', '5461', '6OGjm11', 'fu27', 'ISSCR2017', 'EULAR2017', 'AAIC17',
       'cense2016', 'non-CF', 'mu-opoid', 'ONIS-ANGPT', 'out-of', 'eular2016', 'Lefmk47', '6juOv79', 'WCLC2016', 'EXk61', 'EjOGO57', '4Q16', 'ASCO16', 'cPnY186',
       'hBZiu81','24-week', 'Ex-Dezim', 'rate-to', '8E12', 'edge-Based', 'Ado-trast', 'O0nI10', 'ASH2016', 'RSLDN2016', 'ASCO16', 'Pq3fK45', 'ATS2018', 'MOR208', 'IdiFh13', 'ESHG2016', 'non-infer', 'rGEN-X', 'nkle-reduc', 'EANM2016', 'ICTMC2017', 'DDW17', 'SITC2017' 'WIyR70', 'CCMID2017', 
       'AASLD16', 'TRIMS2016', 'KEYNOTE006', 'TORE-IMI', 'ASCO2018','anti-seizu', 'SITC2017', 'ESHG2017', 'wr4R27', '5126', 'ATS2017', 'BSW17', 'dry-eye', 
        'AACR18', 'ulse-conti', 'IBIT-MESO', 'FECT-HF',  'JAK1', 'JAK2', 'Ph2', 'Ph1', 'Ph3', 'long-term', 'irst-in', 'some-posit', 'Pre-IND', 
        'stol-Myers', 'anti-Tau', 'non-cysti', 'GOOD HEALTH', 'tion-resis', 'ment-exper', 'anti-tuber', 'cebo-contr', 'llow-up', 'well-balan', 'irus-based',
        'ally-advan', 'avel-time', 'top-line', 'ROS1-posit', 'long-actin', 'cult-to', 'non-syste', 'arly-stage', 'rite-up', 'tice-chang', 'ROVE-IT', 'all-cause',
       'uana-Based', 'AP-CBD', 'T-cell', '13-0', 'non-Hodgk', 're-used', 'iRNA-allel', 'Fc-fused', 'mild-to', 'INI1-negat', 'MRI-guide', 'open-label', 're-reead', 'free-rider', 
        'irst-line', 'non-indus', 'TNBC-MERIT', 'enti-D', 'Wu-Xi', 'ex-someb', 'anut-aller', 'Can-Fite', 'phil-deple', 'anti-multi', 'rass-SPIRE', 'abis-based', 'issi-Thode',
        'SIRP-alpha', 'two-trial', 'gram-', '2-trial', 'ngle-arm', 'anti-aging', 'low-dose',
        'apid-fire', 'iota-resto', 'New-', 'Anti-aging', 'well-toler', 'mega-round', 'Site-less', 'mega-round', 'go-ahead', 'long-delay', 'CAR-T', 'lash-and', 'DSP-Visul',
         'ngle-agent', 'itor-in', 'RET-alter',  'late-stage', 'late-stage', 'ngle-cell', 'ONIS-MAPTR', 'post-marke', 'half-life', 'beta-amylo',
        'ncer-fight', 'ET-alter', 'anti-tumor', 'uble-blind', 'irst-time', 'rial-', 'MTL-CEBPA', 'I-bound', 'Mind-contr', 'I-ready', 'mid-stage', 'Mind-contr', 'mega-block', 'High-Perfo',
        'anti-TIM', 'Cancer-fight', 'slash-and', 'nowledge-Based', 'rapid-fire', 'proof-of', 'aghrissi-Thode', 'first-in', 'stration-resis', 'immuno-oncol', 'SiRNA-allel', 'Editor-in', 'Immuno-oncol', 'Single-cell', 'crobiota-resto',
        'First-time', 'IMPROVE-IT', 'placebo-contr', 'first-line', 'program-']

const = 0 #counter

for i in range(len(dataframe)):
    if str(dataframe.loc[i, 'SYNONYMS']) == 'Null': #suggest that if the synonym is already found it should be a correct synonym
        #here are four types of regular expressions
        first = re.findall(r'\w{1,6}\-\d{1,9}', str(dataframe.loc[i, 'TWEET'])) 
        second = re.findall(r'\w{1,9}\d{,1}\-\d{,3}\w{,5}\d{,2}', str(dataframe.loc[i, 'TWEET']))
        third = re.findall(r'\w{,4}\d?\w{1,3}?\-\w{1,4}\-\w{1,7}?\d{1,3}', str(dataframe.loc[i, 'TWEET']))
        fourth = re.findall(r'\w{2,15}\-?\d{2,9}', str(dataframe.loc[i, 'TWEET']))
        
        #conditions depending on that
        #first look at the third regular expression, it is the biggest and the most complicated
        if (len(third) > 0) and (third[0] not in notl):
            res = third
        elif (((len(third) > 0) and (third[0] in notl)) or len(third) == 0)  and (len(second) > 0) and (second[0] not in notl):
            res = second
        elif (((len(third) > 0) and (third[0] in notl)) or len(third) == 0) and (((len(second) > 0) and (second[0] in notl)) or len(second) == 0) and ((len(first) > 0) and (first[0] not in notl)):
            res = first
        else:
            if len(re.findall(r'\w\w\d\-\w\w\w\w', str(dataframe.loc[i, 'TWEET']))) > 0:
                res = re.findall(r'\w\w\d\-\w\w\w\w', str(dataframe.loc[i, 'TWEET']))
            else:
                res = fourth

        if len(res) > 0:
            if (res[0] not in notl) and (len(re.findall('^\w?\d{,4}$', res[0])) == 0): #if the word is not in the list of mess and it's not only a digit word
                years = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022'] #to check one more time that the word is not the name of conference (they usually contain the year)
                check = 0 #checker
                for year in range(len(years)): 
                    if len(re.findall(years[year], res[0])) > 0: #if there is a year in the word
                        check += 1 #increase checker

                if (check == 0) and (dataframe.loc[i, 'SYNONYMS'] == 'Null'): #if the checker is not mentioned and the synonyms cell is free
                    dataframe.at[i, 'SYNONYMS'] = res[0] #add the information
                    dataframe.at[i, 'MOLREGNO'] = 'Not found' 
                    const += 1 # +1 to counter to know how many words were added by regular expressions

In [32]:
print('How many words were matched by regular expressions', const) 

How many words were matched by regular expressions 388


In [33]:
#The cell for information about what was already done
added_by_reg_exp = [] #list of twids of rows that added by regular expressions
fromregexp = 0
regexp_added_indexes = [] #list of indexes of cells that added by regular expressions
inf = 0

for i in range(len(dataframe)):
    if (dataframe.loc[i, 'SYNONYMS'] != 'Null'):
        inf += 1
        lenmatched = (len(indexes_synonyms) - 1)
        helper = 0
        for j in range(len(indexes_synonyms)):
            if str(dataframe.loc[i, 'TWID']) != str(indexes_synonyms[j]):
                helper += 1
                
        if helper == lenmatched:
            added_by_reg_exp.append(dataframe.loc[i, 'TWID'])
            regexp_added_indexes.append(i)
            fromregexp += 1

uniq = 0 #count how many unique twids there are
for i in range(1, len(dataframe)):
    if str(dataframe.loc[i, 'TWID']) != str(dataframe.loc[i - 1, 'TWID']):
        uniq += 1
        
print('The amount of words that were added by regular expressions is', fromregexp)
print('The length of the dataframe is', len(dataframe))
print('The amount of unique tweets is now', uniq)
print('The amount of rows with synonyms is', inf)

The amount of words that were added by regular expressions is 400
The length of the dataframe is 1224
The amount of unique tweets is now 1130
The amount of rows with synonyms is 788


In [34]:
dataframe[:10]

,TWID,TWEET_ID,TWEET,PHASE,N_PHASES,SYNONYMS,MOLREGNO,ACRONYM
0,4,986398045897191424,@PoseidaThera present promising results in ini...,Phase 1,1,P-BCMA-101,Not found,Null
1,16,986219643881885698,Genentech gains a license option to Kineta's a...,Null,0,ACETYLCHOLINE,27812,Null
2,16,986219643881885698,Genentech gains a license option to Kineta's a...,Null,0,ACETYLCHOLINE,195974,Null
3,21,985928258708492288,First-in-human trial with MG7-CART shows effic...,Phase 1,1,MG7-CART,Not found,Null
4,33,985172281734377473,Roche's CpAM modulator RO7049389 well tolerate...,Phase 1,1,RO7049389,Not found,Null
5,40,984750427186126848,Live at #ILC2018 FXR agonist EYP001 from Enyo ...,Phase 1,1,Null,Null,Null
6,65,981919705912004609,Boehringer Ingelheim to progress OSE's SIRP-al...,Null,0,OSE-172,Not found,Null
7,106,977514137046667265,41 patients have been enrolled in the ongoing ...,Phase 2,1,BINIMETINIB,1677014,Null
8,106,977514137046667265,41 patients have been enrolled in the ongoing ...,Phase 2,1,RIBOCICLIB,2039237,Null
9,106,977514137046667265,41 patients have been enrolled in the ongoing ...,Phase 2,1,RIBOCICLIB,1926959,Null


### <font color='green'> Match words that are not matched </font>

In [35]:
not_matched = pd.read_csv('NOT_MATCHED_WORDS.csv') #open a table with words that synonyms' table doesn't have + that were not found by anything else
not_matched = not_matched['DRUGS']

In [36]:
q = 0 #to look how many words were added by this method
for i in range(len(dataframe)):
    if str(dataframe.loc[i, 'SYNONYMS']) == 'Null': #if the synonym cell is empty
        #look in a clean tweet
        tweet = dataframe.loc[i, 'TWEET'].lower().replace('-', ' ').replace('@', ' ').replace('$', ' ').replace('#', ' ').replace('?', ' ').replace('!', ' ').replace('\\', ' ').replace('+', ' ').replace(')', ' ').replace('(', ' ').replace('[', ' ').replace(']', ' ').replace('|', ' ').replace(',', ' ').replace('.', ' ').replace(':', ' ').replace(';', ' ').replace('{', ' ').replace('}', ' ').split()
        for j in range(len(not_matched)): #for the length of the table with not found words
            nm = not_matched.loc[j].lower().replace('-', ' ')
            nm = nm.split() #there can be not only one word, we need to look for every
            x = 0 #counter
            for k in range(len(nm)): #for every word in cell from not_matched table
                if nm[k] in tweet: #if the word is in tweet
                    x += 1 #counter + 1
            if len(nm) == x: #if the amount of words in cells is the same as couner (all words are in tweet)
                dataframe.at[i, 'SYNONYMS'] = not_matched.loc[j].upper() #add the information to synonyms cell
                q += 1 #to look how many words were added by this method
                
print('The amount of synonyms that were added by this table is', q)

The amount of synonyms that were added by this table is 125


In [37]:
dataframe[:20]

,TWID,TWEET_ID,TWEET,PHASE,N_PHASES,SYNONYMS,MOLREGNO,ACRONYM
0,4,986398045897191424,@PoseidaThera present promising results in ini...,Phase 1,1,P-BCMA-101,Not found,Null
1,16,986219643881885698,Genentech gains a license option to Kineta's a...,Null,0,ACETYLCHOLINE,27812,Null
2,16,986219643881885698,Genentech gains a license option to Kineta's a...,Null,0,ACETYLCHOLINE,195974,Null
3,21,985928258708492288,First-in-human trial with MG7-CART shows effic...,Phase 1,1,MG7-CART,Not found,Null
4,33,985172281734377473,Roche's CpAM modulator RO7049389 well tolerate...,Phase 1,1,RO7049389,Not found,Null
5,40,984750427186126848,Live at #ILC2018 FXR agonist EYP001 from Enyo ...,Phase 1,1,EYP001,Null,Null
6,65,981919705912004609,Boehringer Ingelheim to progress OSE's SIRP-al...,Null,0,OSE-172,Not found,Null
7,106,977514137046667265,41 patients have been enrolled in the ongoing ...,Phase 2,1,BINIMETINIB,1677014,Null
8,106,977514137046667265,41 patients have been enrolled in the ongoing ...,Phase 2,1,RIBOCICLIB,2039237,Null
9,106,977514137046667265,41 patients have been enrolled in the ongoing ...,Phase 2,1,RIBOCICLIB,1926959,Null


In [38]:
# to look how many words were added by all these matchings
a = 0 #counter
for i in range(len(dataframe)):
    if (str(dataframe.loc[i, 'SYNONYMS']) == 'Null') and (str(dataframe.loc[i, 'ACRONYM']) == 'Null'): 
        #if there is no information about synonym or acronym
        a += 1 #counter +1
        
print(a, len(dataframe))

318 1224


In [39]:
print('The amount of matched tweets is', (len(dataframe) - a)/len(dataframe) * 100, '%')

The amount of matched tweets is 74.01960784313727 %


### <font color='green'> Deleting not relevant tweets </font>

<font color='purple'> **Before** running this part it is **neccessary** to check which tweets are not with any information. While I was running it it showed me that it's a mess or something around this. The ones that were not a mess I added to the list with *not matched words* </font>

In [ ]:
#you can check by this cell
for i in range(len(dataframe)):
    if (str(dataframe.loc[i, 'SYNONYMS']) == 'Null') and (str(dataframe.loc[i, 'ACRONYM']) == 'Null'): #if there is no information about synonym or acronym
        print(i, dataframe.loc[i. 'TWEET']) #print the tweet
        print('______________') #print a separator

In [40]:
delete_tweets = [] #list of indexes of tweets we need to delete
empty = 0 #counter
for i in range(len(dataframe)):
    #if there is no information about synonym or acronym
    if str(dataframe.loc[i, 'SYNONYMS']) == 'Null' and str(dataframe.loc[i, 'ACRONYM']) == 'Null':
        delete_tweets.append(i) #append index to the list
        empty += 1 #add 1 to counter
    
print('The amount of tweets without information about synonyms is', empty)

The amount of tweets without information about synonyms is 290


In [41]:
dataframe = dataframe.drop(dataframe.index[delete_tweets]) #delete one time all the tweets by indexes
dataframe.index = np.arange(len(dataframe)) #update the indexing

print('The length of tweets with found synonyms is', len(dataframe))

The length of tweets with found synonyms is 934


In [42]:
#check how many unique tweets are still in the table
twids_c = 0 #counter
for i in range(1, len(dataframe)):
    if str(dataframe.loc[i, 'TWID']) != str(dataframe.loc[i - 1, 'TWID']): 
        twids_c += 1
        
print('The amount of unique tweets is', twids_c)

The amount of unique tweets is 840


In [5]:
dataframe[:5]

,TWID,TWEET_ID,TWEET,PHASE,N_PHASES,SYNONYMS,MOLREGNO,ACRONYM
0,4,986398045897191424,@PoseidaThera present promising results in ini...,Phase 1,1,P-BCMA-101,Not found,Null
1,16,986219643881885698,Genentech gains a license option to Kineta's a...,Null,0,ACETYLCHOLINE,27812,Null
2,16,986219643881885698,Genentech gains a license option to Kineta's a...,Null,0,ACETYLCHOLINE,195974,Null
3,21,985928258708492288,First-in-human trial with MG7-CART shows effic...,Phase 1,1,MG7-CART,Not found,Null
4,33,985172281734377473,Roche's CpAM modulator RO7049389 well tolerate...,Phase 1,1,RO7049389,Not found,Null


## <font color='green'> Outcome from the tweet </font>

In [6]:
outcomes = pd.read_csv(r'Outcome.csv', encoding='utf8') #open the table with outcomes
outcomes[:4]

,REG_EXP,RESULT
0,shows efficacy,SUCCESS
1,promising results,SUCCESS
2,well tolerated,SUCCESS
3,safe well tolerated,SUCCESS


In [7]:
len(outcomes)

392

Matching the outcomes with tweets

In [8]:
ww_table = dataframe #copy to work with

for i in range(len(ww_table)): 
    #clean tweet
    ww_table.loc[i, 'TWEET'] =  ww_table.loc[i, 'TWEET'].replace(',', '').replace('.', '').replace(';', '').replace(':', '')
    
for i in range(len(ww_table)): #for the word in dataframe
    for j in range(len(outcomes)): #for the word in table with outcomes
        out = outcomes.loc[j, 'REG_EXP'].lower().split() #make a list of words in cell with outcome
        tweet = ww_table.loc[i, 'TWEET'].lower().split() #make a list of words in cell with tweet
        
        counter = 0 #counter
        for k in range(len(out)):
            if out[k] in tweet: #if the word is in tweet
                counter += 1 #add 1 to counter
        if counter == len(out): #if all words in tweet
            dataframe.at[i, 'OUTCOME'] = outcomes.loc[j, 'RESULT'] #add information about final outcome
            dataframe.at[i, 'MATCHED_OUTCOME'] = outcomes.loc[j, 'REG_EXP'] #add information about the reason of decision
            
            
dataframe = dataframe.drop_duplicates() #delete duplicates
dataframe = dataframe.fillna('Null') #fill empty cells with 'Null'

In [9]:
dataframe

,TWID,TWEET_ID,TWEET,PHASE,N_PHASES,SYNONYMS,MOLREGNO,ACRONYM,OUTCOME,MATCHED_OUTCOME
0,4,986398045897191424,@PoseidaThera present promising results in ini...,Phase 1,1,P-BCMA-101,Not found,Null,SUCCESS,promising results
1,16,986219643881885698,Genentech gains a license option to Kineta's a...,Null,0,ACETYLCHOLINE,27812,Null,SUCCESS,gains a license
2,16,986219643881885698,Genentech gains a license option to Kineta's a...,Null,0,ACETYLCHOLINE,195974,Null,SUCCESS,gains a license
3,21,985928258708492288,First-in-human trial with MG7-CART shows effic...,Phase 1,1,MG7-CART,Not found,Null,SUCCESS,shows safety
4,33,985172281734377473,Roche's CpAM modulator RO7049389 well tolerate...,Phase 1,1,RO7049389,Not found,Null,SUCCESS,well tolerated
5,40,984750427186126848,Live at #ILC2018 FXR agonist EYP001 from Enyo ...,Phase 1,1,EYP001,Null,Null,IN PROGRESS,In phase live at
6,65,981919705912004609,Boehringer Ingelheim to progress OSE's SIRP-al...,Null,0,OSE-172,Not found,Null,IN PROGRESS,to progress for
7,106,977514137046667265,41 patients have been enrolled in the ongoing ...,Phase 2,1,BINIMETINIB,1677014,Null,IN PROGRESS,in the ongoing phase
8,106,977514137046667265,41 patients have been enrolled in the ongoing ...,Phase 2,1,RIBOCICLIB,2039237,Null,IN PROGRESS,in the ongoing phase
9,106,977514137046667265,41 patients have been enrolled in the ongoing ...,Phase 2,1,RIBOCICLIB,1926959,Null,IN PROGRESS,in the ongoing phase


<font color='purple'> One more time to **check** tweets for relevance because there is still some mess we could not use </font>

In [10]:
c = 0 #counter
for i in range(len(dataframe)):
    if str(dataframe.loc[i, 'OUTCOME']) == 'Null': #if the outcome is empty
        print(dataframe.loc[i, 'TWEET']) #print tweet
        c += 1 #counter +1
        print('_______________________') #print a separator

Ex-Dezima CEO co-founds NASH startup flying into Phase IIB with 25 million
@BrittanyMeiling
https//tco/OVO41lOayo
_______________________
#3
UPDATED Watch out Allergan Revance's wrinkle-reducer beats Botox in PhIII trials
-- $AGN responds to the challenge
https//tco/oFbEvbb5gr
_______________________
Watch out Allergan Revance's wrinkle-reducer beats Botox in PhIII trials
$RVNC +36%
https//tco/oFbEvbb5gr
_______________________
RT @MaxJacobsEdison I love this reminder of the dangers of subgroup analysis  Apparently in a 17000 patient trial Aspirin had a negativ
_______________________
#2
Innate shares crushed on second big setback for Bristol-Myers partnered lirilumab
$BMY $IPH
https//tco/aslqF0wWkK
_______________________
We'll have a truncated report out in just a bit With an annual special
Innate shares crushed on second big setback for Bristol-Myers partnered lirilumab
$BMY Euronext IPH
https//tco/aslqF0wWkK
_______________________
from weekend bureau
Merck KGaA impresses on OA
htt

In [11]:
print('The amount of not relevent tweets now is', c)

The amount of not relevent tweets now is 74


In [12]:
#delete these rows
mess = [] #the list with indexes of not relevant rows
for i in range(len(dataframe)):
    if str(dataframe.loc[i, 'OUTCOME']) == 'Null' or str(dataframe.loc[i, 'OUTCOME']) == 'MESS': #if there is no information or there is information that it's a mess
        mess.append(i) #add the index to the list
        
dataframe = dataframe.drop(dataframe.index[mess]) #delete these rows
dataframe.index = np.arange(len(dataframe)) #update the indexing

In [13]:
print('Now the length of main dataframe is', len(dataframe))

Now the length of main dataframe is 838


# <font color='green'>  II. Working with tables from termite </font>

## <font color='green'> Prepare a termite table </font>

In [91]:
#open tables
termite_main_init = pd.read_csv('TERMITE_TABLE.csv', encoding='utf8', names = ['TWID', 'ENTITY_TYPE', 'ENTITY_ID', 'ENTITY_NAME', 'MATCHED_SYN', 'MATCHED_POS', 'SCORE', 'NONAMBIG'])
termite_suppl_init = pd.read_csv(r'TRIALS_TERMITE.csv', encoding='utf8', names=['NCT_ID', 'ENTITY_TYPE', 'ENTITY_ID', 'ENTITY_NAME', 'MATCH_SYN', 'MATCH_POS', 'SCORE', 'NONAMBIG'])

#create the frames of new ones
termite_main = pd.DataFrame(columns=['TWID', 'DRUG', 'DRUG_ID', 'DRUG_SCORE', 'DRUG_NONAMBIG', 'DRUG_NCT_ID', 'DRUG_MATCHED', 'COMPANY', 'COMPANY_ID', 'COMPANY_SCORE', 'COMPANY_NONAMBIG', 'COMPANY_MATCHED', 'INDICATION', 'INDICATION_ID', 'INDICATION_SCORE', 'INDICATION_NONAMBIG', 'INDICATION_NCT_ID', 'INDICATION_MATCHED'])
termite_suppl = pd.DataFrame(columns=['INDICATION_ID', 'INDICATION_NAME', 'INDICATION_NCT_ID', 'INDICATION_SCORE', 'INDICATION_NONAMBIG', 'DRUG_ID', 'DRUG_NAME', 'DRUG_NCT_ID', 'DRUG_SCORE', 'DRUG_NONAMBIG'])

Turn upside down the main table: 

In [92]:
#for the first raw:
termite_main.at[0, 'TWID'] = termite_main_init.iloc[0, 0] #add the information about the first tweed
if termite_main_init.iloc[0, 1] == 'COMPANY': #if the ENTITY_TYPE is 'company' add information to the company
    termite_main.at[0, 'COMPANY'] = termite_main_init.iloc[0, 3]
    termite_main.at[0, 'COMPANY_ID'] = termite_main_init.iloc[0, 2]
    termite_main.at[0, 'COMPANY_SCORE'] = termite_main_init.iloc[0, 6]
    termite_main.at[0, 'COMPANY_NONAMBIG'] = termite_main_init.iloc[0, 7]
    termite_main.at[0, 'COMPANY_MATCHED'] = termite_main_init.loc[0, 'MATCHED_SYN']

elif termite_main_init.iloc[0, 1] == 'DRUG': #if the ENTITY_TYPE is 'drug' add information to the company
    termite_main.at[0, 'DRUG'] = termite_main_init.iloc[0, 3]
    termite_main.at[0, 'DRUG_ID'] = termite_main_init.iloc[0, 2]
    termite_main.at[0, 'DRUG_SCORE'] = termite_main_init.iloc[0, 6]
    termite_main.at[0, 'DRUG_NONAMBIG'] = termite_main_init.iloc[0, 7]
    termite_main.at[0, 'DRUG_MATCHED'] = termite_main_init.loc[0, 'MATCHED_SYN']
    
elif termite_main_init.iloc[0, 1] == 'INDICATION': #if the ENTITY_TYPE is 'indication' add information to the company
    termite_main.at[0, 'INDICATION'] = termite_main_init.iloc[0, 3]
    termite_main.at[0, 'INDICATION_ID'] = termite_main_init.iloc[0, 2]
    termite_main.at[0, 'INDICATION_SCORE'] = termite_main_init.iloc[0, 6]
    termite_main.at[0, 'INDICATION_NONAMBIG'] = termite_main_init.iloc[0, 7]
    termite_main.at[0, 'INDICATION_MATCHED'] = termite_main_init.loc[0, 'MATCHED_SYN']

#for others raws:
p = 0 #position in new_termite table
for i in range(1, len(termite_main_init)):
    if termite_main_init.iloc[i, 0] != termite_main.iloc[p, 0]: #if twid is other
        p += 1 #come to next position
        termite_main.at[p, 'TWID'] = termite_main_init.iloc[i, 0] #add twid to the table
        if termite_main_init.iloc[i, 1] == 'COMPANY': #if the ENTITY_TYPE is 'company' add information to the company
            termite_main.at[p, 'COMPANY'] = termite_main_init.iloc[i, 3]
            termite_main.at[p, 'COMPANY_ID'] = termite_main_init.iloc[i, 2]
            termite_main.at[p, 'COMPANY_SCORE'] = termite_main_init.iloc[i, 6]
            termite_main.at[p, 'COMPANY_NONAMBIG'] = termite_main_init.iloc[i, 7]
            termite_main.at[p, 'COMPANY_MATCHED'] = termite_main_init.loc[i, 'MATCHED_SYN']

        elif termite_main_init.iloc[i, 1] == 'DRUG': #if the ENTITY_TYPE is 'drug' add information to the company
            termite_main.at[p, 'DRUG'] = termite_main_init.iloc[i, 3]
            termite_main.at[p, 'DRUG_ID'] = termite_main_init.iloc[i, 2]
            termite_main.at[p, 'DRUG_SCORE'] = termite_main_init.iloc[i, 6]
            termite_main.at[p, 'DRUG_NONAMBIG'] = termite_main_init.iloc[i, 7]
            termite_main.at[p, 'DRUG_MATCHED'] = termite_main_init.loc[i, 'MATCHED_SYN']
    
        elif termite_main_init.iloc[i, 1] == 'INDICATION': #if the ENTITY_TYPE is 'indication' add information to the company
            termite_main.at[p, 'INDICATION'] = termite_main_init.iloc[i, 3]
            termite_main.at[p, 'INDICATION_ID'] = termite_main_init.iloc[i, 2]
            termite_main.at[p, 'INDICATION_SCORE'] = termite_main_init.iloc[i, 6]
            termite_main.at[p, 'INDICATION_NONAMBIG'] = termite_main_init.iloc[i, 7]
            termite_main.at[p, 'INDICATION_MATCHED'] = termite_main_init.loc[i, 'MATCHED_SYN']
    else:
        if termite_main_init.iloc[i, 1] == 'COMPANY': #if the ENTITY_TYPE is 'company' add information to the company
            termite_main.at[p, 'COMPANY'] = termite_main_init.iloc[i, 3]
            termite_main.at[p, 'COMPANY_ID'] = termite_main_init.iloc[i, 2]
            termite_main.at[p, 'COMPANY_SCORE'] = termite_main_init.iloc[i, 6]
            termite_main.at[p, 'COMPANY_NONAMBIG'] = termite_main_init.iloc[i, 7]
            termite_main.at[p, 'COMPANY_MATCHED'] = termite_main_init.loc[i, 'MATCHED_SYN']

        elif termite_main_init.iloc[i, 1] == 'DRUG': #if the ENTITY_TYPE is 'drug' add information to the company
            termite_main.at[p, 'DRUG'] = termite_main_init.iloc[i, 3]
            termite_main.at[p, 'DRUG_ID'] = termite_main_init.iloc[i, 2]
            termite_main.at[p, 'DRUG_SCORE'] = termite_main_init.iloc[i, 6]
            termite_main.at[p, 'DRUG_NONAMBIG'] = termite_main_init.iloc[i, 7]
            termite_main.at[p, 'DRUG_MATCHED'] = termite_main_init.loc[i, 'MATCHED_SYN']
    
        elif termite_main_init.iloc[i, 1] == 'INDICATION': #if the ENTITY_TYPE is 'indication' add information to the company
            termite_main.at[p, 'INDICATION'] = termite_main_init.iloc[i, 3]
            termite_main.at[p, 'INDICATION_ID'] = termite_main_init.iloc[i, 2] 
            termite_main.at[p, 'INDICATION_SCORE'] = termite_main_init.iloc[i, 6]
            termite_main.at[p, 'INDICATION_NONAMBIG'] = termite_main_init.iloc[i, 7]
            termite_main.at[p, 'INDICATION_MATCHED'] = termite_main_init.loc[i, 'MATCHED_SYN']
            
termite_main = termite_main.fillna('Null') #fill in empty cells with 'Null'
termite_main.index = np.arange(len(termite_main)) #update indexing

In [93]:
termite_main[:5]

,TWID,DRUG,DRUG_ID,DRUG_SCORE,DRUG_NONAMBIG,DRUG_NCT_ID,DRUG_MATCHED,COMPANY,COMPANY_ID,COMPANY_SCORE,COMPANY_NONAMBIG,COMPANY_MATCHED,INDICATION,INDICATION_ID,INDICATION_SCORE,INDICATION_NONAMBIG,INDICATION_NCT_ID,INDICATION_MATCHED
0,368,Null,Null,Null,Null,Null,Null,AstraZeneca,COMP82,1,1,AstraZeneca,Null,Null,Null,Null,Null,Null
1,369,Macrilen,SDG00531,1,1,Null,Macrilen,Aeterna Zentaris,COMP25,1,1,Aeterna Zentaris,Null,Null,Null,Null,Null,Null
2,373,Null,Null,Null,Null,Null,Null,Roche,COMP642,1,1,Roche,Chronic Progressive Multiple Sclerosis,D020528,1,1,Null,primary progressive multiple sclerosis
3,374,Null,Null,Null,Null,Null,Null,Shire Pharmaceuticals,COMP705,1,0,Shire,Hemophilia A,D006467,1,1,Null,haemophilia A
4,376,Null,Null,Null,Null,Null,Null,Pharmaxis,COMP91241,1,1,Pharmaxis,Diabetic Retinopathy,D003930,1,1,Null,diabetic retinopathy


Turn upside down the second table

In [94]:
pos = 0 #position in termite_suppl

for i in range(len(termite_suppl_init) - 1): 
    if termite_suppl_init.loc[i, 'ENTITY_TYPE'] == 'INDICATION': #if the ENTITY_TYPE is indication
        #add information about indication
        termite_suppl.at[pos, 'INDICATION_ID'] = termite_suppl_init.loc[i, 'ENTITY_ID']
        termite_suppl.at[pos, 'INDICATION_NAME'] = termite_suppl_init.loc[i, 'ENTITY_NAME']
        termite_suppl.at[pos, 'INDICATION_NCT_ID'] = termite_suppl_init.loc[i, 'NCT_ID']
        termite_suppl.at[pos, 'INDICATION_SCORE'] = termite_suppl_init.loc[i, 'SCORE']
        termite_suppl.at[pos, 'INDICATION_NONAMBIG'] = termite_suppl_init.loc[i, 'NONAMBIG']
        

    elif termite_suppl_init.loc[i, 'ENTITY_TYPE'] == 'DRUG': #if the ENTITY_TYPE is drug
        #add information about drug
        termite_suppl.at[pos, 'DRUG_ID'] = termite_suppl_init.loc[i, 'ENTITY_ID']
        termite_suppl.at[pos, 'DRUG_NAME'] = termite_suppl_init.loc[i, 'ENTITY_NAME']
        termite_suppl.at[pos, 'DRUG_NCT_ID'] = termite_suppl_init.loc[i, 'NCT_ID']
        termite_suppl.at[pos, 'DRUG_SCORE'] = termite_suppl_init.loc[i, 'SCORE']
        termite_suppl.at[pos, 'DRUG_NONAMBIG'] = termite_suppl_init.loc[i, 'NONAMBIG']
        
    if termite_suppl_init.loc[i + 1, 'NCT_ID'] != termite_suppl_init.loc[i, 'NCT_ID']: #if rows are not referred to the same nct_id
        pos += 1 #make one step in table
        
termite_suppl = termite_suppl.fillna('Null') #fill in the empty cells with 'Null'
termite_suppl.index = np.arange(len(termite_suppl)) #update the indexing

In [95]:
termite_suppl[:5]

,INDICATION_ID,INDICATION_NAME,INDICATION_NCT_ID,INDICATION_SCORE,INDICATION_NONAMBIG,DRUG_ID,DRUG_NAME,DRUG_NCT_ID,DRUG_SCORE,DRUG_NONAMBIG
0,D010003,Osteoarthritis,NCT00000425,1,1,Null,Null,Null,Null,Null
1,D010024,Osteoporosis,NCT00000426,1,1,Null,Null,Null,Null,Null
2,D010024,Osteoporosis,NCT00000427,1,1,CHEMBL870,Alendronate Sodium,NCT00000427,1,1
3,D005356,Fibromyalgia,NCT00000428,1,1,CHEMBL629,Amitriptyline,NCT00000428,1,1
4,D001171,Juvenile Arthritis,NCT00000429,1,1,Null,Null,Null,Null,Null


Fill in the main table with the information from the supplementary table. If there is information about the drug in main table but there is no the information about the indication, add the information from the supplementary table about indication in table.

In [96]:
termite = termite_main #make a copy of initial table to work with it, it will be a final termite table

In [97]:
#add the columns for result of filling in new information
termite['RESULT_INDICATION'] = 'Null'
termite['RESULT_DRUG'] = 'Null'

#update everything
termite.index = np.arange(len(termite))
termite_suppl.index = np.arange(len(termite_suppl))

#the main part of this adding
for i in range(len(termite)): 
    if termite.loc[i, 'DRUG'] != 'Null': #if there is information about drug in initial table
        for j in range(len(termite_suppl)):
            #if there is such drug in supplementary table and in initial there is no indication
            if (str(termite.loc[i, 'DRUG']).lower() == str(termite_suppl.loc[j, 'DRUG_NAME']).lower()) and (termite_suppl.loc[j, 'INDICATION_NAME'] != 'Null'):
                #fill in the information about indication in main termite table
                termite.at[i, 'INDICATION_ID'] = termite_suppl.loc[j, 'INDICATION_ID']
                termite.at[i, 'INDICATION'] = termite_suppl.loc[j, 'INDICATION_NAME']
                termite.at[i, 'INDICATION_NCT_ID'] = termite_suppl.loc[j, 'INDICATION_NCT_ID']
                termite.at[i, 'INDICATION_SCORE'] = termite_suppl.loc[j, 'INDICATION_SCORE']
                termite.at[i, 'INDICATION_NONAMBIG']= termite_suppl.loc[j, 'INDICATION_NONAMBIG']
                termite.at[i, 'RESULT_DRUG'] = 'Found'
                termite.at[i, 'RESULT_INDICATION'] = 'Filled'

                
#add columns with NCT_ID and Type of NCT_ID
termite['NCT_ID'] = 'Null'
termite['TYPE_NCT_ID'] = 'Null'

#fill in the information about NCT_ID
for i in range(len(termite)):
    if termite.loc[i, 'DRUG_NCT_ID'] != 'Null':
        termite.at[i, 'NCT_ID'] = termite.loc[i, 'DRUG_NCT_ID']
        termite.at[i, 'TYPE_NCT_ID'] = 'Drug'
    else:
        if termite.loc[i, 'INDICATION_NCT_ID'] != 'Null':
            termite.at[i, 'NCT_ID'] = termite.loc[i, 'INDICATION_NCT_ID']
            termite.at[i, 'TYPE_NCT_ID'] = 'Indication'  

KeyboardInterrupt: 

In [ ]:
term = termite #make a new table to work with

In [ ]:
term[:3]

In [ ]:
#delete columns that are not relevant for further matchings
del term['DRUG']
del term['DRUG_ID']
del term['DRUG_SCORE']
del term['DRUG_NONAMBIG']
del term['DRUG_NCT_ID']
del term['DRUG_MATCHED']

In [ ]:
term[:8]

In [ ]:
term.to_csv('termite_new.csv', sep='\t', encoding='utf8')

## <font color='green'> Join the termite table and the main table </font>

In [18]:
#check the order of columns
dataframe = dataframe[['TWID', 'TWEET_ID', 'TWEET', 'SYNONYMS', 'MOLREGNO', 'ACRONYM', 'PHASE', 'N_PHASES', 'OUTCOME', 'MATCHED_OUTCOME']]

In [19]:
dataframe[:3]

,TWID,TWEET_ID,TWEET,SYNONYMS,MOLREGNO,ACRONYM,PHASE,N_PHASES,OUTCOME,MATCHED_OUTCOME
0,4,986398045897191424,@PoseidaThera present promising results in ini...,P-BCMA-101,Not found,Null,Phase 1,1,SUCCESS,promising results
1,16,986219643881885698,Genentech gains a license option to Kineta's a...,ACETYLCHOLINE,27812,Null,Null,0,SUCCESS,gains a license
2,16,986219643881885698,Genentech gains a license option to Kineta's a...,ACETYLCHOLINE,195974,Null,Null,0,SUCCESS,gains a license


In [20]:
#convert the type of cell to do the correct matching
dataframe['TWID'] = dataframe['TWID'].astype(np.int64)
term['TWID'] = term['TWID'].astype(np.int64)

In [21]:
uni = 0 #count how many unique twids there are
for i in range(1, len(dataframe)):
    if str(dataframe.loc[i, 'TWID']) != str(dataframe.loc[i - 1, 'TWID']):
        uni += 1
        
print('The amount of unique tweets now is', uni)

The amount of unique tweets now is 750


In [22]:
dataframe = pd.merge(dataframe, term, how='left', on='TWID') #connect the main dataframe and the termite table by 'TWID'
dataframe.index = np.arange(len(dataframe)) #update the indexing
dataframe = dataframe.fillna('Null') #fill in empty places with 'Null'
print(len(dataframe)) #length of the dataframe now is
dataframe[:2]

841


,TWID,TWEET_ID,TWEET,SYNONYMS,MOLREGNO,ACRONYM,PHASE,N_PHASES,OUTCOME,MATCHED_OUTCOME,COMPANY,COMPANY_ID,COMPANY_SCORE,COMPANY_NONAMBIG,COMPANY_MATCHED,INDICATION,INDICATION_ID,INDICATION_SCORE,INDICATION_NONAMBIG,INDICATION_NCT_ID
0,4,986398045897191424,@PoseidaThera present promising results in ini...,P-BCMA-101,Not found,Null,Phase 1,1,SUCCESS,promising results,Null,Null,Null,Null,Null,Multiple Myeloma,D009101,1,1,Null
1,16,986219643881885698,Genentech gains a license option to Kineta's a...,ACETYLCHOLINE,27812,Null,Null,0,SUCCESS,gains a license,Kineta®,COMP92965,1,1,AstraZeneca,Chronic Pain,D059350,1,1,Null


# <font color='green'> III. Adding the information from other tables. </font>

## <font color='green'> Exctracting NCT_ID! </font>

In [23]:
#open the list with interventions (Paula's table)
interventions = pd.read_csv('INTERVENTIONS.csv', names=['NCT_ID', 'INTERVENTION_TYPE', 'INTERVENTION_NAME', 
                                                        'DESCRIPTIONS', 'OTHER_NAME', 'ARM_GROUP_LABEL'])

interventions[:2]

,NCT_ID,INTERVENTION_TYPE,INTERVENTION_NAME,DESCRIPTIONS,OTHER_NAME,ARM_GROUP_LABEL
0,NCT00003593,Drug,cytarabine,Given IT,cytosine arabinoside;AraC;Cytosar;NSC #063878,Arm A: TMD patients only
1,NCT00003593,Drug,cytarabine,Given IT,cytosine arabinoside;AraC;Cytosar;NSC #063878,Arm B: AML/MDS patients only


In [24]:
dataframe_copy = dataframe #make a copy of main dataframe

In [25]:
dataframe = dataframe_copy

In [26]:
#Parallelization of the process, making 6 chunks to work with them on different kernels
#Dividing main table to 6 parts

a = dataframe[0 : len(dataframe) // 6]
a.index = np.arange(len(a))

b = dataframe[len(dataframe) // 6 : len(dataframe) // 3]
b.index = np.arange(len(b))

c = dataframe[len(dataframe) // 3 : len(dataframe) // 2]
c.index = np.arange(len(c))

d = dataframe[len(dataframe) // 2 : (2 * len(dataframe)) // 3]
d.index = np.arange(len(d))

e = dataframe[(2 * len(dataframe)) // 3 : (5 * len(dataframe)) // 6]
e.index = np.arange(len(e))

f = dataframe[(5 * len(dataframe)) // 6 : len(dataframe)]
f.index = np.arange(len(f)) 

#list of chunks of the working table that will be run by function
chunks = [a, b, c, d, e, f]

#number of chunks = number of kernels that is going to use during the process

In [27]:
def look_interventions(chunk): #function itself
    for i in range(len(chunk)): #for every word from the part of main table
        for j in range(len(interventions)): #for all the words from table with interventions
            if str(chunk.loc[i, 'SYNONYMS']) != 'Null': #if there is synonym
                #try to find it in 'INTERVENTION_NAME' or 'OTHER_NAME' column from interventions table
                res_1 = re.findall(str(chunk.loc[i, 'SYNONYMS']).lower().replace('-', ''), str(interventions.loc[j,'INTERVENTION_NAME']).lower().replace('-', ''))
                res_2 = re.findall(str(chunk.loc[i, 'SYNONYMS']).lower().replace('-', ''), str(interventions.loc[j,'OTHER_NAME']).lower().replace('-', ''))
                if len(res_1) > 0 or len(res_2) > 0: #if found
                    chunk.at[i, 'SYNONYMS_NCT_ID'] = interventions.loc[j, 'NCT_ID'] #add the nct_id to a new column to main dataset
            elif str(chunk.loc[i, 'ACRONYM']) != 'Null': #if there is acronym
                #try to find it in 'INTERVENTION_NAME' or 'OTHER_NAME' column from interventions table
                res_1 = re.findall(str(chunk.loc[i, 'ACRONYM']).lower().replace('-', ''), str(interventions.loc[j,'INTERVENTION_NAME']).lower().replace('-', ''))
                res_2 = re.findall(str(chunk.loc[i, 'ACRONYM']).lower().replace('-', ''), str(interventions.loc[j,'OTHER_NAME']).lower().replace('-', ''))
                if len(res_1) > 0 or len(res_2) > 0: #if found
                    chunk.at[i, 'SYNONYMS_NCT_ID'] = interventions.loc[j, 'NCT_ID'] #add the information (nct_id)
    return chunk

if __name__ == '__main__': #constructor for parallezation
    p = Pool(6) #use 6 kernels to work on them
    look = p.map(look_interventions, chunks) #function to do what I need
    print(look) 
    
dataframe = dataframe[0:0] #create a frame for the main table
for l in range(len(look)):
    dataframe = dataframe.append(look[l]) #add all updated parts to the main table
    
dataframe.index = np.arange(len(dataframe)) #update the indexes
dataframe = dataframe.fillna('Null') #fill in empty cells

[     TWID            TWEET_ID  \
0       4  986398045897191424   
1      16  986219643881885698   
2      16  986219643881885698   
3      21  985928258708492288   
4      33  985172281734377473   
5      40  984750427186126848   
6      65  981919705912004609   
7     106  977514137046667265   
8     106  977514137046667265   
9     106  977514137046667265   
10    116  977140733919547392   
11    140  976673091500814336   
12    141  976672839821611008   
13    177  975811214906478592   
14    182  975694542572802049   
15    183  975690650514489344   
16    184  975689033983647744   
17    195  975490429943721986   
18    203  975374696249163776   
19    208  975080041397792769   
20    219  974573688434692096   
21    222  973893417821528065   
22    226  973584058985918465   
23    226  973584058985918465   
24    226  973584058985918465   
25    231  973515394765000704   
26    258  970630221702615040   
27    260  970593118444752898   
28    260  970593118444752898   
29    274

/anaconda3/lib/python3.5/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
Process ForkPoolWorker-6:
Process ForkPoolWorker-5:
Process ForkPoolWorker-3:
Process ForkPoolWorker-2:
Process ForkPoolWorker-4:
Process ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda3/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/anaconda3/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    sel

In [58]:
dataframe[:2]

,TWID,TWEET_ID,TWEET,SYNONYMS,MOLREGNO,ACRONYM,SYNONYMS_NCT_ID,PHASE,N_PHASES,OUTCOME,MATCHED_OUTCOME,COMPANY,COMPANY_ID,COMPANY_MATCHED,COMPANY_NONAMBIG,COMPANY_SCORE,INDICATION,INDICATION_ID,INDICATION_NONAMBIG,INDICATION_SCORE
0,4,986398045897191424,@PoseidaThera present promising results in ini...,P-BCMA-101,Not found,Null,NCT03288493,Phase 1,1,SUCCESS,promising results,Null,Null,Null,Null,Null,Multiple Myeloma,D009101,1,1
1,16,986219643881885698,Genentech gains a license option to Kineta's a...,ACETYLCHOLINE,27812,Null,NCT03446612,Null,0,SUCCESS,gains a license,Kineta®,COMP92965,AstraZeneca,1,1,Chronic Pain,D059350,1,1


In [28]:
#order of columns
dataframe = dataframe[['TWID', 'TWEET_ID', 'TWEET', 'SYNONYMS', 'MOLREGNO', 'ACRONYM', 'SYNONYMS_NCT_ID', 
                       'PHASE', 'N_PHASES', 'OUTCOME', 'MATCHED_OUTCOME', 'COMPANY', 'COMPANY_ID',
                       'COMPANY_MATCHED', 'COMPANY_NONAMBIG', 'COMPANY_SCORE', 'INDICATION', 'INDICATION_ID',
                       'INDICATION_NONAMBIG', 'INDICATION_SCORE']]

In [29]:
#cell to have some new updated information about what was made with the table
found = 0 #counter for found words
not_found = 0 #counter for not found words
dataframe.index = np.arange(len(dataframe)) #update the indexing

for i in range(len(dataframe)):
    if str(dataframe.loc[i, 'SYNONYMS_NCT_ID']) != 'Null': #if there is information about nct_id
        found += 1 #add 1 to the first counter
    else: #if no information
        not_found += 1 #add the information to the other counter
        
print('During the work with intervention names were found', found, 'words')
print('Not found', not_found, 'words')
print('The length of the table is', len(dataframe))

During the work with intervention names were found 657 words
Not found 184 words
The length of the table is 841


In [30]:
dataframe[:4]

,TWID,TWEET_ID,TWEET,SYNONYMS,MOLREGNO,ACRONYM,SYNONYMS_NCT_ID,PHASE,N_PHASES,OUTCOME,MATCHED_OUTCOME,COMPANY,COMPANY_ID,COMPANY_MATCHED,COMPANY_NONAMBIG,COMPANY_SCORE,INDICATION,INDICATION_ID,INDICATION_NONAMBIG,INDICATION_SCORE
0,4,986398045897191424,@PoseidaThera present promising results in ini...,P-BCMA-101,Not found,Null,NCT03288493,Phase 1,1,SUCCESS,promising results,Null,Null,Null,Null,Null,Multiple Myeloma,D009101,1,1
1,16,986219643881885698,Genentech gains a license option to Kineta's a...,ACETYLCHOLINE,27812,Null,NCT03446612,Null,0,SUCCESS,gains a license,Kineta®,COMP92965,AstraZeneca,1,1,Chronic Pain,D059350,1,1
2,16,986219643881885698,Genentech gains a license option to Kineta's a...,ACETYLCHOLINE,195974,Null,NCT03446612,Null,0,SUCCESS,gains a license,Kineta®,COMP92965,AstraZeneca,1,1,Chronic Pain,D059350,1,1
3,21,985928258708492288,First-in-human trial with MG7-CART shows effic...,MG7-CART,Not found,Null,NCT02862704,Phase 1,1,SUCCESS,shows safety,Null,Null,Null,Null,Null,Rectal Neoplasms,D012004,1,1


## <font color='green'> Matching with other datasets: *INTERVENTIONS, EFO2NAME, NCT_ID2MOLREGNO, NCTID_CONDITION_EFO* </font>

In [31]:
#open datasets I need
interventions = pd.read_csv('INTERVENTIONS.csv', names=['NCT_ID', 'INTERVENTION_TYPE', 'INTERVENTION_NAME', 
                                                        'DESCRIPTIONS', 'OTHER_NAME', 'ARM_GROUP_LABEL'])

efo2name = pd.read_csv('EFO2NAME.csv', names=['EFO_ID', 'NAME_TERM'])

NCT_ID2MOLREGNO = pd.read_csv('NCT_ID2MOLREGNO.csv', names=['NCT_ID', 'MOLREGNO', 'INTERVENTION_NAME',
                                                            'ARM_GROUP_TYPE', 'ARM_GROUP_LABEL', 'FOR_DICTIONARY'])

NCTID_CONDITION_EFO = pd.read_csv('NCTID_CONDITION_EFO.csv', names=['NCT_ID', 'EFO_ID', 'CONDITION', 'REDUNDANT', 
                                                                    'FOR_AUTOMATIC_MAPPING', 'IS_INDICATION'])


In [250]:
df = dataframe #make a copy
df = df.rename(columns={'SYNONYMS_NCT_ID':'NCT_ID'}) #rename columns
df = df.rename(columns={'MOLREGNO': 'MOLREGNO_1'})

In [251]:
len(df)

841

In [252]:
#Now we work only with rows that have NCT_ID
clean = df #copy

nt = [] #list with indexes of not relevant rows

for i in range(len(clean)):
    if str(clean.loc[i, 'NCT_ID']) == 'Null': #if there is no information about NCT_ID
        nt.append(i) #add the index

#clean = clean.drop(clean.index[nt])
clean.index = np.arange(len(clean))

In [253]:
len(clean)

841

In [254]:
df = pd.merge(clean, interventions, how='left', on='NCT_ID') #connect dataframe with interventions table

df = df.fillna('Null') #fill in empty cells
del df['ARM_GROUP_LABEL'] #delete 'ARM_GROUP_LABEL' column
df = df.drop_duplicates() #delete duplicates
df.index = np.arange(len(df)) #update indexing
print(len(df))

1958


In [255]:
df

,TWID,TWEET_ID,TWEET,SYNONYMS,MOLREGNO_1,ACRONYM,NCT_ID,PHASE,N_PHASES,OUTCOME,...,COMPANY_NONAMBIG,COMPANY_SCORE,INDICATION,INDICATION_ID,INDICATION_NONAMBIG,INDICATION_SCORE,INTERVENTION_TYPE,INTERVENTION_NAME,DESCRIPTIONS,OTHER_NAME
0,4,986398045897191424,@PoseidaThera present promising results in ini...,P-BCMA-101,Not found,Null,NCT03288493,Phase 1,1,SUCCESS,...,Null,Null,Multiple Myeloma,D009101,1,1,Biological,P-BCMA-101 CAR-T cells,"P-BCMA-101 is an autologous, principally Tscm,...",Null
1,16,986219643881885698,Genentech gains a license option to Kineta's a...,ACETYLCHOLINE,27812,Null,NCT03446612,Null,0,SUCCESS,...,1,1,Chronic Pain,D059350,1,1,Drug,L-N-monomethyl arginine citrate (L-NMMA),L-N-monomethyl arginine citrate will be used a...,Null
2,16,986219643881885698,Genentech gains a license option to Kineta's a...,ACETYLCHOLINE,27812,Null,NCT03446612,Null,0,SUCCESS,...,1,1,Chronic Pain,D059350,1,1,Drug,Daprodustat,"Daprodustat will be available as 1 mg, 2 mg an...",Null
3,16,986219643881885698,Genentech gains a license option to Kineta's a...,ACETYLCHOLINE,27812,Null,NCT03446612,Null,0,SUCCESS,...,1,1,Chronic Pain,D059350,1,1,Drug,Darbepoetin alfa (Aranesp),Darbepoetin alfa will be given as solution for...,Null
4,16,986219643881885698,Genentech gains a license option to Kineta's a...,ACETYLCHOLINE,27812,Null,NCT03446612,Null,0,SUCCESS,...,1,1,Chronic Pain,D059350,1,1,Drug,Acetylcholine,Acetylcholine will be used as a challenge agen...,Null
5,16,986219643881885698,Genentech gains a license option to Kineta's a...,ACETYLCHOLINE,27812,Null,NCT03446612,Null,0,SUCCESS,...,1,1,Chronic Pain,D059350,1,1,Drug,Sodium nitroprusside,Sodium nitroprusside will be used as a challen...,Null
6,16,986219643881885698,Genentech gains a license option to Kineta's a...,ACETYLCHOLINE,195974,Null,NCT03446612,Null,0,SUCCESS,...,1,1,Chronic Pain,D059350,1,1,Drug,L-N-monomethyl arginine citrate (L-NMMA),L-N-monomethyl arginine citrate will be used a...,Null
7,16,986219643881885698,Genentech gains a license option to Kineta's a...,ACETYLCHOLINE,195974,Null,NCT03446612,Null,0,SUCCESS,...,1,1,Chronic Pain,D059350,1,1,Drug,Daprodustat,"Daprodustat will be available as 1 mg, 2 mg an...",Null
8,16,986219643881885698,Genentech gains a license option to Kineta's a...,ACETYLCHOLINE,195974,Null,NCT03446612,Null,0,SUCCESS,...,1,1,Chronic Pain,D059350,1,1,Drug,Darbepoetin alfa (Aranesp),Darbepoetin alfa will be given as solution for...,Null
9,16,986219643881885698,Genentech gains a license option to Kineta's a...,ACETYLCHOLINE,195974,Null,NCT03446612,Null,0,SUCCESS,...,1,1,Chronic Pain,D059350,1,1,Drug,Acetylcholine,Acetylcholine will be used as a challenge agen...,Null


There appeared rows that contain information about the same NCT_ID but the other drug. So this is why we should delete not relevant ones.

In [256]:
df['INDIC_INTERVENTIONS'] = 'False' #new column for the result

In [257]:
#Parallelization of the process, making 6 chunks to work with them on different kernels
#Make 6 parts of the working table

a = df[0 : len(df) // 6]
a.index = np.arange(len(a))

b = df[len(df) // 6 : len(df) // 3]
b.index = np.arange(len(b))

c = df[len(df) // 3 : len(df) // 2]
c.index = np.arange(len(c))

d = df[len(df) // 2 : (2 * len(df)) // 3]
d.index = np.arange(len(d))

e = df[(2 * len(df)) // 3 : (5 * len(df)) // 6]
e.index = np.arange(len(e))

f = df[(5 * len(df)) // 6 : len(df)]
f.index = np.arange(len(f)) 

#list of chunks of the working table that will be run by function
chunks = [a, b, c, d, e, f]

#number of chunks = number of kernels that is going to use during the process

In [258]:
def true_interv(chunk):
    for i in range(len(chunk)):
        #if synonym in the 'INTERVENTION_NAME' or in the 'OTHER_NAME'
        res_1 = re.findall(str(chunk.loc[i, 'SYNONYMS']).lower().replace('-', ''), chunk.loc[i, 'INTERVENTION_NAME'].lower().replace('-', ''))
        res_2 = re.findall(str(chunk.loc[i, 'SYNONYMS']).lower().replace('-', ''), chunk.loc[i, 'OTHER_NAME'].lower().replace('-', ''))
        if len(res_1) > 0 or len(res_2) > 0: #if yes that is the row we need
            chunk.at[i, 'INDIC_INTERVENTIONS'] = 'True' #true matching
    return chunk
                   
if __name__ == '__main__': #constructor
    p = Pool(6)
    listik = p.map(true_interv, chunks) #main function
    print(listik) 
    
df = df[0:0] #a frame for the further work
for l in range(len(listik)):
    df = df.append(listik[l]) #add updated chunks to the main table
    
df.index = np.arange(len(df)) #update the indexing

[     TWID            TWEET_ID  \
0       4  986398045897191424   
1      16  986219643881885698   
2      16  986219643881885698   
3      16  986219643881885698   
4      16  986219643881885698   
5      16  986219643881885698   
6      16  986219643881885698   
7      16  986219643881885698   
8      16  986219643881885698   
9      16  986219643881885698   
10     16  986219643881885698   
11     21  985928258708492288   
12     33  985172281734377473   
13     33  985172281734377473   
14     40  984750427186126848   
15     40  984750427186126848   
16     65  981919705912004609   
17    106  977514137046667265   
18    106  977514137046667265   
19    106  977514137046667265   
20    106  977514137046667265   
21    106  977514137046667265   
22    106  977514137046667265   
23    106  977514137046667265   
24    106  977514137046667265   
25    116  977140733919547392   
26    140  976673091500814336   
27    141  976672839821611008   
28    177  975811214906478592   
29    177

In [259]:
#the table with information about rows copies (with the same twid)
#we will need it not to delete all the information about nct_id's
twids = pd.DataFrame(columns=['TWID', 'TIMES'])

twids.at[0, 'TWID'] = df.iloc[0, 0] #add the first twid
num = 0

for i in range(1, len(df)):
    if df.iloc[i, 0] != twids.iloc[num, 0]:
        num += 1
        twids.at[num, 'TWID'] = df.iloc[i, 0]         
    
i = 0
j = 0
counter = 0
while i != len(df):
    if df.iloc[i, 0] == twids.iloc[j, 0]:
        i += 1
        counter += 1
    elif df.iloc[i, 0] != twids.iloc[j, 0]:
        twids.at[j, 'TIMES'] = counter
        i += 1
        j += 1
        counter = 1 
        
twids[:2]

,TWID,TIMES
0,4,1
1,16,10


In [260]:
#This is cell to clean the ones that are not correct
cleaner = [] #list with indexes to clean
for i in range(len(df)):
    if str(df.loc[i, 'INDIC_INTERVENTIONS']) != 'True': #if the indicator is not true
        for j in range(len(twids)):
            if str(twids.loc[j, 'TWID']) == str(df.loc[i, 'TWID']): #find twid in the supplementary table
                if twids.loc[j, 'TIMES'] > 1: #and if the there is more than one copy of this row
                    cleaner.append(i) #add the index
        
df = df.drop(df.index[cleaner]) #delete not relevant ones
df = df.drop_duplicates() #delete duplicates
df.index = np.arange(len(df)) #update the indexing
del df['INDIC_INTERVENTIONS'] #now we so not need this column with indicator, delete it
print(len(df)) 

946


In [261]:
df[:2]

,TWID,TWEET_ID,TWEET,SYNONYMS,MOLREGNO_1,ACRONYM,NCT_ID,PHASE,N_PHASES,OUTCOME,...,COMPANY_NONAMBIG,COMPANY_SCORE,INDICATION,INDICATION_ID,INDICATION_NONAMBIG,INDICATION_SCORE,INTERVENTION_TYPE,INTERVENTION_NAME,DESCRIPTIONS,OTHER_NAME
0,4,986398045897191424,@PoseidaThera present promising results in ini...,P-BCMA-101,Not found,Null,NCT03288493,Phase 1,1,SUCCESS,...,Null,Null,Multiple Myeloma,D009101,1,1,Biological,P-BCMA-101 CAR-T cells,"P-BCMA-101 is an autologous, principally Tscm,...",Null
1,16,986219643881885698,Genentech gains a license option to Kineta's a...,ACETYLCHOLINE,27812,Null,NCT03446612,Null,0,SUCCESS,...,1,1,Chronic Pain,D059350,1,1,Drug,Acetylcholine,Acetylcholine will be used as a challenge agen...,Null


In [262]:
df = pd.merge(df, NCTID_CONDITION_EFO, how='left', on='NCT_ID') #match with the NCTID_CONDITION_EFO table by NCT_ID
df = df.drop_duplicates() #delete full duplicates
df.index = np.arange(len(df)) #update indexing
del df['REDUNDANT'] #delete not needed columns
del df['FOR_AUTOMATIC_MAPPING']
del df['IS_INDICATION']

len(df)

1449

In [263]:
df = pd.merge(df, NCT_ID2MOLREGNO, how='left', on='NCT_ID') #match with the NCT_ID2MOLREGNO table by NCT_ID
df = df.fillna('Null') #fill in empty cells
del df['ARM_GROUP_TYPE'] #delete not needed colums
del df['ARM_GROUP_LABEL']
del df['FOR_DICTIONARY']
del df['INTERVENTION_NAME_y']
df = df.drop_duplicates() #drop duplicates
df.index = np.arange(len(df)) #update the indexing

In [264]:
len(df) #a lot of duplicates appeared because of the different conditions

2754

In [265]:
df[:2]

,TWID,TWEET_ID,TWEET,SYNONYMS,MOLREGNO_1,ACRONYM,NCT_ID,PHASE,N_PHASES,OUTCOME,...,INDICATION_ID,INDICATION_NONAMBIG,INDICATION_SCORE,INTERVENTION_TYPE,INTERVENTION_NAME_x,DESCRIPTIONS,OTHER_NAME,EFO_ID,CONDITION,MOLREGNO
0,4,986398045897191424,@PoseidaThera present promising results in ini...,P-BCMA-101,Not found,Null,NCT03288493,Phase 1,1,SUCCESS,...,D009101,1,1,Biological,P-BCMA-101 CAR-T cells,"P-BCMA-101 is an autologous, principally Tscm,...",Null,EFO:0001378,Multiple Myeloma,Null
1,16,986219643881885698,Genentech gains a license option to Kineta's a...,ACETYLCHOLINE,27812,Null,NCT03446612,Null,0,SUCCESS,...,D059350,1,1,Drug,Acetylcholine,Acetylcholine will be used as a challenge agen...,Null,EFO:0004272,Anaemia,226851


In [266]:
uni = 0 #count how many unique twids there are
df.index = np.arange(len(df))
for i in range(1, len(df)):
    if str(df.loc[i, 'TWID']) != str(df.loc[i - 1, 'TWID']):
        uni += 1
        
print('The amount of unique tweets is', uni) #585

The amount of unique tweets is 746


In [267]:
df = pd.merge(df, efo2name, how='left', on='EFO_ID') #match with the efo2name by EFO_ID
df = df.drop_duplicates() #drop duplicates
df.index = np.arange(len(df)) #update indexing
len(df)

2754

In [268]:
#Now we have two columns with molregnos and we can compare the relevance of tweets
h = 0 #counter
df.index = np.arange(len(df))
dif_molregno = [] #list with the indexes of rows that are not correct
for i in range(len(df)):
    if str(df.loc[i, 'MOLREGNO_1']) != 'Not found' and str(df.loc[i, 'MOLREGNO_1']) != 'Null': #if there is MOLREGNO extracted by synonyms' table
        if str(df.loc[i, 'MOLREGNO_1']) != str(df.loc[i, 'MOLREGNO']): #if it is not as the MOLREGNO from indications table
            dif_molregno.append(i) #add the index to the list
            h += 1 #add 1 to a counter
            
for i in range(len(dif_molregno)): #look which ones are not correct
    print(df.loc[dif_molregno[i]])
    print('_____')

TWID                                                                  16
TWEET_ID                                              986219643881885698
TWEET                  Genentech gains a license option to Kineta's a...
SYNONYMS                                                   ACETYLCHOLINE
MOLREGNO_1                                                         27812
ACRONYM                                                             Null
NCT_ID                                                       NCT03446612
PHASE                                                               Null
N_PHASES                                                               0
OUTCOME                                                          SUCCESS
MATCHED_OUTCOME                                          gains a license
COMPANY                                                          Kineta®
COMPANY_ID                                                     COMP92965
COMPANY_MATCHED                                    

Name: 116, dtype: object
_____
TWID                                                                 481
TWEET_ID                                              939597072801681408
TWEET                  Acalabrutinib (100 mg bid) demonstrates promis...
SYNONYMS                                                   ACALABRUTINIB
MOLREGNO_1                                                       2039319
ACRONYM                                                             Null
NCT_ID                                                       NCT02296918
PHASE                                                            Phase 2
N_PHASES                                                               1
OUTCOME                                                          SUCCESS
MATCHED_OUTCOME                                    demonstrates efficacy
COMPANY                                                             Null
COMPANY_ID                                                          Null
COMPANY_MATCHED     

_____
TWID                                                                1285
TWEET_ID                                              871476260161155072
TWEET                  Daratumumab+carfilzomib+lenalidomide+dexametha...
SYNONYMS                                                     DARATUMUMAB
MOLREGNO_1                                                       1121878
ACRONYM                                                             Null
NCT_ID                                                       NCT03289299
PHASE                                                               Null
N_PHASES                                                               0
OUTCOME                                                          SUCCESS
MATCHED_OUTCOME                                           well tolerated
COMPANY                                                             Null
COMPANY_ID                                                          Null
COMPANY_MATCHED                              

Name: 845, dtype: object
_____
TWID                                                                1705
TWEET_ID                                              842712473887100928
TWEET                  Preliminary toxicity results presented from ph...
SYNONYMS                                                       CETUXIMAB
MOLREGNO_1                                                        675579
ACRONYM                                                             Null
NCT_ID                                                       NCT03391843
PHASE                                                            Phase 3
N_PHASES                                                               1
OUTCOME                                                             FAIL
MATCHED_OUTCOME                               toxicity results presented
COMPANY                                                             Null
COMPANY_ID                                                          Null
COMPANY_MATCHED     

TWID                                                                1969
TWEET_ID                                              817343622941020160
TWEET                  Novaliq's CyclAsol (ophthalmic ciclosporin) re...
SYNONYMS                                                     CICLOSPORIN
MOLREGNO_1                                                         72873
ACRONYM                                                             Null
NCT_ID                                                       NCT01527045
PHASE                                                            Phase 2
N_PHASES                                                               1
OUTCOME                                                          SUCCESS
MATCHED_OUTCOME                     reduces corneal fluorescein staining
COMPANY                                                             Null
COMPANY_ID                                                          Null
COMPANY_MATCHED                                    

...

_____
TWID                                                                2749
TWEET_ID                                              759933566792835072
TWEET                  Edge Therapeutics starts phase III trial of EG...
SYNONYMS                                                      NIMODIPINE
MOLREGNO_1                                                        365248
ACRONYM                                                             Null
NCT_ID                                                       NCT03150524
PHASE                                                            Phase 3
N_PHASES                                                               2
OUTCOME                                                      IN PROGRESS
MATCHED_OUTCOME                                       starts phase trial
COMPANY                                                Edge Therapeutics
COMPANY_ID                                                     COMP92360
COMPANY_MATCHED                              

_____
TWID                                                                2964
TWEET_ID                                              741567279050149888
TWEET                  Sirukumab efficacy in phase 3 study in active ...
SYNONYMS                                                       SIRUKUMAB
MOLREGNO_1                                                       1121942
ACRONYM                                                             Null
NCT_ID                                                       NCT00718718
PHASE                                                            Phase 3
N_PHASES                                                               1
OUTCOME                                                      IN PROGRESS
MATCHED_OUTCOME                                         In phase live at
COMPANY                                                             Null
COMPANY_ID                                                          Null
COMPANY_MATCHED                              

Name: 2085, dtype: object
_____
TWID                                                                4697
TWEET_ID                                              956508440326746112
TWEET                  GW on fire PhIII epilepsy data signals success...
SYNONYMS                                                             CBD
MOLREGNO_1                                                        318390
ACRONYM                                                             Null
NCT_ID                                                       NCT03382561
PHASE                                                            Phase 3
N_PHASES                                                               2
OUTCOME                                                              N/I
MATCHED_OUTCOME                                               First data
COMPANY                                                             Null
COMPANY_ID                                                          Null
COMPANY_MATCHED    

Name: 2286, dtype: object
_____
TWID                                                               19430
TWEET_ID                                              991798269239062528
TWEET                  Study by @TheCrick investigators may explain w...
SYNONYMS                                                       LAPATINIB
MOLREGNO_1                                                        675130
ACRONYM                                                             Null
NCT_ID                                                       NCT00404066
PHASE                                                               Null
N_PHASES                                                               0
OUTCOME                                                             FAIL
MATCHED_OUTCOME                                               why failed
COMPANY                                                             Null
COMPANY_ID                                                          Null
COMPANY_MATCHED    

_____
TWID                                                               51714
TWEET_ID                                             1003650693763301376
TWEET                  Durvalumab + cisplatin + pemetrexed was active...
SYNONYMS                                                      PEMETREXED
MOLREGNO_1                                                        374008
ACRONYM                                                             Null
NCT_ID                                                       NCT02039674
PHASE                                                            Phase 2
N_PHASES                                                               2
OUTCOME                                                          SUCCESS
MATCHED_OUTCOME                                       Tolerable in trial
COMPANY                                                             Null
COMPANY_ID                                                          Null
COMPANY_MATCHED                              

In [269]:
df = df.drop(df.index[dif_molregno]) #delete these ones
df.index = np.arange(len(df)) #update the indexing
len(df)

1656

<font color='blue'> Add information about MeSH: </font>

In [270]:
efo2mesh = pd.read_csv('EFO2MESH.csv', names=['MESH_ID', 'EFO_ID', 'INVALID_LINK', 'MESH_ID_COPY'])
del efo2mesh['INVALID_LINK']
del efo2mesh['MESH_ID_COPY']
efo2mesh[:2]

,MESH_ID,EFO_ID
0,D002277,EFO:0000313
1,D060050,EFO:0003913


In [271]:
df = pd.merge(df, efo2mesh, on='EFO_ID')
df = df.fillna('Null')
df = df.drop_duplicates()
df[:2]

,TWID,TWEET_ID,TWEET,SYNONYMS,MOLREGNO_1,ACRONYM,NCT_ID,PHASE,N_PHASES,OUTCOME,...,INDICATION_SCORE,INTERVENTION_TYPE,INTERVENTION_NAME_x,DESCRIPTIONS,OTHER_NAME,EFO_ID,CONDITION,MOLREGNO,NAME_TERM,MESH_ID
0,4,986398045897191424,@PoseidaThera present promising results in ini...,P-BCMA-101,Not found,Null,NCT03288493,Phase 1,1,SUCCESS,...,1,Biological,P-BCMA-101 CAR-T cells,"P-BCMA-101 is an autologous, principally Tscm,...",Null,EFO:0001378,Multiple Myeloma,Null,multiple myeloma,D009101
1,1285,871476260161155072,Daratumumab+carfilzomib+lenalidomide+dexametha...,LENALIDOMIDE,63649,Null,NCT01863550,Null,0,SUCCESS,...,1,Drug,Lenalidomide,Given PO,CC-5013;CC5013;CDC 501;Revlimid,EFO:0001378,Plasma Cell Myeloma,63649,multiple myeloma,D009101


In [272]:
uni = 0 #count how many unique twids there are
for i in range(1, len(df)):
    if str(df.loc[i, 'TWID']) != str(df.loc[i - 1, 'TWID']):
        uni += 1
        
print('The amount of unique tweets is', uni) #585

The amount of unique tweets is 665


In [273]:
len(df)

1439

## <font color='green'> Matching with original clinical trials table </font>

In [274]:
#open the clinicaltrials table
clinical_trials = pd.read_csv('CLINICALTRIALS_ORIGINAL.csv', names =['NCT_ID', 'ORG_STUDY_ID', 'OVERSIGHT_INFO_AUTHORITY', 
                                                                     'OVERALL_STATUS', 'WHY_STOPPED', 'PHASES',
                                                                     'STUDY_TYPE', 'STUDY_DESIGN', 'NUMBER_OF_ARMS',
                                                                     'CONDITION_CT', 'START_DATE', 'COMPLETION_DATE', 
                                                                     'COMPLETION_DATA_TYPE', 'PRIMARY_COMPLETION_DATE',
                                                                     'PRIMARY_COMPLETION_DATA_TYPE',
                                                                     'VERIFICATION_DATE', 'LASTCHANGED_DATE', 
                                                                     'FIRSTRECEIVED_DATE', 'FIRSTRECEIVED_RESULTS_DATE', 
                                                                     'INTERVENTION_BROWSE_MESH_TERM', 
                                                                     'CONDITION_BROWSE_MESH_TERM', 'RESULTS', 
                                                                     'BRIEF_TITLE', 'OFFICIAL_TITLE', 'LEAD_SPONSOR', 
                                                                     'STUDY_FIRST_SUBMITTED', 'LAST_UPDATE_POSTED', 
                                                                     'LAST_UPDATE_POSTED_TYPE', 'RESULTS_FIRST_POSTED', 
                                                                     'RESULTS_FIRST_POSTED_TYPE', 'ACRONYMS'],
                             index_col=False)


len(clinical_trials)

151533

In [275]:
#deleting not needed columns
del clinical_trials['OVERSIGHT_INFO_AUTHORITY']
del clinical_trials['WHY_STOPPED']
del clinical_trials['NUMBER_OF_ARMS']
del clinical_trials['PRIMARY_COMPLETION_DATE']
del clinical_trials['PRIMARY_COMPLETION_DATA_TYPE']
del clinical_trials['COMPLETION_DATA_TYPE']
del clinical_trials['BRIEF_TITLE']
del clinical_trials['STUDY_FIRST_SUBMITTED']
del clinical_trials['LAST_UPDATE_POSTED']
del clinical_trials['LAST_UPDATE_POSTED_TYPE']
del clinical_trials['RESULTS_FIRST_POSTED']
del clinical_trials['RESULTS_FIRST_POSTED_TYPE']
del clinical_trials['CONDITION_BROWSE_MESH_TERM']
del clinical_trials['INTERVENTION_BROWSE_MESH_TERM']
del clinical_trials['FIRSTRECEIVED_RESULTS_DATE']
del clinical_trials['FIRSTRECEIVED_DATE']
del clinical_trials['LASTCHANGED_DATE']
del clinical_trials['VERIFICATION_DATE']

In [276]:
clinical_trials = clinical_trials.fillna('Null') #fill empty cells with 'Null'
df = pd.merge(df, clinical_trials, how='left', on='NCT_ID') #match main dataframe with clinicaltrials by NCT_ID
df.index = np.arange(len(df)) #update indexing
len(df)

1470

In [277]:
df[:3]

,TWID,TWEET_ID,TWEET,SYNONYMS,MOLREGNO_1,ACRONYM,NCT_ID,PHASE,N_PHASES,OUTCOME,...,PHASES,STUDY_TYPE,STUDY_DESIGN,CONDITION_CT,START_DATE,COMPLETION_DATE,RESULTS,OFFICIAL_TITLE,LEAD_SPONSOR,ACRONYMS
0,4,986398045897191424,@PoseidaThera present promising results in ini...,P-BCMA-101,Not found,Null,NCT03288493,Phase 1,1,SUCCESS,...,Phase 1,Interventional,"Intervention Model: Single Group Assignment, I...",Multiple Myeloma,Null,Null,Null,"Open-Label, Multicenter, Single Ascending Dose...","Poseida Therapeutics, Inc.",Null
1,1285,871476260161155072,Daratumumab+carfilzomib+lenalidomide+dexametha...,LENALIDOMIDE,63649,Null,NCT01863550,Null,0,SUCCESS,...,Phase 3,Interventional,"Allocation: Randomized, Intervention Model: Pa...",Plasma Cell Myeloma,2013-11-01 00:00:00,Null,Null,"Randomized Phase III Trial of Bortezomib, LENa...",ECOG-ACRIN Cancer Research Group,Null
2,1285,871476260161155072,Daratumumab+carfilzomib+lenalidomide+dexametha...,DARATUMUMAB,1121878,Null,NCT03289299,Null,0,SUCCESS,...,Phase 2,Interventional,"Intervention Model: Single Group Assignment, I...",Smoldering Multiple Myeloma,Null,Null,Null,Aggressive Smoldering Curative Approach Evalua...,International Myeloma Foundation,ASCENT


In [278]:
df = df.fillna('Null') #fill empty cells with 'Null'
df = df.drop_duplicates() #delete all the suplicates
df.index = np.arange(len(df)) #update indexes
df.to_csv('res_table.csv', sep='\t', encoding='utf8') #save to csv

In [279]:
df[:2]

,TWID,TWEET_ID,TWEET,SYNONYMS,MOLREGNO_1,ACRONYM,NCT_ID,PHASE,N_PHASES,OUTCOME,...,PHASES,STUDY_TYPE,STUDY_DESIGN,CONDITION_CT,START_DATE,COMPLETION_DATE,RESULTS,OFFICIAL_TITLE,LEAD_SPONSOR,ACRONYMS
0,4,986398045897191424,@PoseidaThera present promising results in ini...,P-BCMA-101,Not found,Null,NCT03288493,Phase 1,1,SUCCESS,...,Phase 1,Interventional,"Intervention Model: Single Group Assignment, I...",Multiple Myeloma,Null,Null,Null,"Open-Label, Multicenter, Single Ascending Dose...","Poseida Therapeutics, Inc.",Null
1,1285,871476260161155072,Daratumumab+carfilzomib+lenalidomide+dexametha...,LENALIDOMIDE,63649,Null,NCT01863550,Null,0,SUCCESS,...,Phase 3,Interventional,"Allocation: Randomized, Intervention Model: Pa...",Plasma Cell Myeloma,2013-11-01 00:00:00,Null,Null,"Randomized Phase III Trial of Bortezomib, LENa...",ECOG-ACRIN Cancer Research Group,Null


## <font color='green'> Calculations </font>

Here would be a comparison of table.

In [280]:
db = df #make a copy
db[:2]

,TWID,TWEET_ID,TWEET,SYNONYMS,MOLREGNO_1,ACRONYM,NCT_ID,PHASE,N_PHASES,OUTCOME,...,PHASES,STUDY_TYPE,STUDY_DESIGN,CONDITION_CT,START_DATE,COMPLETION_DATE,RESULTS,OFFICIAL_TITLE,LEAD_SPONSOR,ACRONYMS
0,4,986398045897191424,@PoseidaThera present promising results in ini...,P-BCMA-101,Not found,Null,NCT03288493,Phase 1,1,SUCCESS,...,Phase 1,Interventional,"Intervention Model: Single Group Assignment, I...",Multiple Myeloma,Null,Null,Null,"Open-Label, Multicenter, Single Ascending Dose...","Poseida Therapeutics, Inc.",Null
1,1285,871476260161155072,Daratumumab+carfilzomib+lenalidomide+dexametha...,LENALIDOMIDE,63649,Null,NCT01863550,Null,0,SUCCESS,...,Phase 3,Interventional,"Allocation: Randomized, Intervention Model: Pa...",Plasma Cell Myeloma,2013-11-01 00:00:00,Null,Null,"Randomized Phase III Trial of Bortezomib, LENa...",ECOG-ACRIN Cancer Research Group,Null


In [281]:
del db['MOLREGNO_1']
del db['ACRONYM']
del db['NCT_ID']
del db['OUTCOME']
del db['STUDY_TYPE']
del db['STUDY_DESIGN']
del db['START_DATE']
del db['COMPLETION_DATE']
del db['RESULTS']
del db['ACRONYMS']
del db['MATCHED_OUTCOME']
del db['COMPANY_ID']
del db['COMPANY_MATCHED']
del db['COMPANY_NONAMBIG']
del db['COMPANY_SCORE']
del db['INDICATION_NONAMBIG']
del db['INDICATION_SCORE']
del db['INTERVENTION_TYPE']
del db['INTERVENTION_NAME_x']
del db['DESCRIPTIONS']
del db['OTHER_NAME']
del db['EFO_ID']
del db['MOLREGNO']
del db['NAME_TERM']
del db['ORG_STUDY_ID']
del db['OVERALL_STATUS']
db[:2]

,TWID,TWEET_ID,TWEET,SYNONYMS,PHASE,N_PHASES,COMPANY,INDICATION,INDICATION_ID,CONDITION,MESH_ID,PHASES,CONDITION_CT,OFFICIAL_TITLE,LEAD_SPONSOR
0,4,986398045897191424,@PoseidaThera present promising results in ini...,P-BCMA-101,Phase 1,1,Null,Multiple Myeloma,D009101,Multiple Myeloma,D009101,Phase 1,Multiple Myeloma,"Open-Label, Multicenter, Single Ascending Dose...","Poseida Therapeutics, Inc."
1,1285,871476260161155072,Daratumumab+carfilzomib+lenalidomide+dexametha...,LENALIDOMIDE,Null,0,Null,Multiple Myeloma,D009101,Plasma Cell Myeloma,D009101,Phase 3,Plasma Cell Myeloma,"Randomized Phase III Trial of Bortezomib, LENa...",ECOG-ACRIN Cancer Research Group


In [197]:
keys = pd.read_csv('Key_indications.csv', encoding='utf8')

,Keys,Nan
0,oma cancer emia neoplasm myelodysplastic syndr...,NaN


In [282]:
#create needed columns
db['COMPARE_INDICATION'] = 'False'
db['COMPARE_COMPANY'] = 'False'
db['COMPARE_PHASE'] = 'False'

porter = nltk.PorterStemmer() #for stemmer

for i in range(len(db)):
    #clear tweets to match 
    db.at[i, 'INDICATION'] = str(db.loc[i, 'INDICATION']).replace(';', '').replace(',', '').replace(':', '').replace('\'s', '').replace('.', '')
    
    #clearing for CONDITION
    db.at[i, 'CONDITION'] = str(db.loc[i, 'CONDITION']).replace(';', '').replace(',', '').replace(':', '').replace('\'s', '').replace('.', '')
    
    #simple matching
    ind = str(db.loc[i, 'INDICATION']).split(' ')
    ch_indication = [] #the list to check
    for j in range(len(ind)):            
        comp_ind = re.findall(ind[j].lower(), str(db.loc[i, 'CONDITION']).lower())
        if (len(comp_ind) > 0):
            ch_indication.append(1) #if at least one time the word is in the condition add 1
        else:
            ch_indication.append(0)
        
    if 1 in ch_indication: #if at least one word in list set true
        db.at[i, 'COMPARE_INDICATION'] = 'True'
    elif 1 not in ch_indication: #if not - set false
        db.at[i, 'COMPARE_INDICATION'] = 'False'
        
        
    #comparing using stemmer, only main parts of words    
    stem_x = [porter.stem(t) for t in str(db.loc[i, 'INDICATION']).split(' ')]
    stem_y = [porter.stem(t) for t in str(db.loc[i, 'CONDITION']).split(' ')]
    
    x = 0
    for k in range(len(stem_x)): #look for every word after stemming in condition
        if stem_x[k] in stem_y:
            x += 1
    if x > 0:
        db.at[i, 'COMPARE_INDICATION'] = 'True'
    
    #comparing indications using the key dictionary (ex. 'oma' -> 'cancer')
    if str(db.loc[i, 'COMPARE_INDICATION']) != 'True':
        for j in range(len(keys)):
            keys_j = keys.loc[j, 'Keys'].split(' ')
            res_x = 0
            res_y = 0
            for k in range(len(keys_j)):
                key_x = re.findall(str(keys_j[k]).lower(), str(db.loc[i, 'INDICATION']).lower())
                key_y = re.findall(str(keys_j[k]).lower(), str(db.loc[i, 'CONDITION']).lower())
                if len(key_x) > 0: 
                    res_x += 1
                if len(key_y) > 0:
                    res_y += 1
                
            if res_x > 0 and res_y > 0:
                db.at[i, 'COMPARE_INDICATION'] = 'True'
    
    #check indication by mesh id:
    if str(db.loc[i, 'INDICATION_ID']) == str(db.loc[i, 'MESH_ID']):
         db.at[i, 'COMPARE_INDICATION'] = 'True'
        
    #compare company
    res_comp = re.findall(str(db.loc[i, 'COMPANY']).lower(), str(db.loc[i, 'LEAD_SPONSOR']).lower())
    if len(res_comp) > 0:
        db.at[i, 'COMPARE_COMPANY'] = 'True'
            
            
    #compare phase
    res_ph = re.findall(str(db.loc[i, 'PHASE']).lower(), str(db.loc[i, 'PHASES']).lower())
    if len(res_ph) > 0:
        db.at[i, 'COMPARE_PHASE'] = 'True'

In [283]:
same_phase = 0
same_ind = 0
same_comp = 0
for i in range(len(db)):
    if str(db.loc[i, 'COMPARE_PHASE']) == 'True':
        same_phase += 1
    
    if str(db.loc[i, 'COMPARE_INDICATION']) == 'True':
        same_ind += 1
        
    if str(db.loc[i, 'COMPARE_COMPANY']) == 'True':
        same_comp +=1
        
print('Rows with same phases -', same_phase)
print('Rows with the same companies -', same_comp)
print('Rows with the same indications -', same_ind)
print('Length of the dataset is', len(db))

Rows with same phases - 558
Rows with the same companies - 255
Rows with the same indications - 674
Length of the dataset is 1439
